# CultureSense — Longitudinal Clinical Hypothesis Engine
## Kaggle HAI-DEF Competition Submission

[![MedGemma](https://img.shields.io/badge/MedGemma-4b--it-blue)](https://huggingface.co/google/medgemma-4b-it)
[![Safety](https://img.shields.io/badge/Safety-Non--Diagnostic-green)]()
[![Mode](https://img.shields.io/badge/Mode-Patient%20%2B%20Clinician-purple)]()

> **CultureSense** processes 2–3 sequential urine or stool culture lab reports and produces
> structured, **non-diagnostic** interpretations through two distinct output modes.
> MedGemma handles natural language generation from already-structured inputs.
> Deterministic rules handle all temporal signal extraction.

---

## Architecture

```mermaid
flowchart TD
    A["[1] Raw Report Ingestion\nList[str] 2-3 free-text culture reports"] --> B
    B["[2] Structured Extraction Layer\nextract_structured_data() → CultureReport"] --> C
    C["[3] Temporal Comparison Engine\nanalyze_trend() → TrendResult"] --> D
    D["[4] Hypothesis Update Layer\ngenerate_hypothesis() → HypothesisResult\nconfidence [0.0–0.95]"] --> E
    E["[5] MedGemma Reasoning Layer\ncall_medgemma(structured_payload, mode) → str\nModes: patient | clinician"] --> F
    F["[6] Structured Safe Output Renderer\nrender_output() → FormattedOutput\nPatient: explanation + questions\nClinician: trajectory + confidence + flags"]

    style A fill:#e8f4f8
    style B fill:#d4edda
    style C fill:#d4edda
    style D fill:#fff3cd
    style E fill:#f8d7da
    style F fill:#e8f4f8
```

**Key safety invariant:** Raw report text is NEVER forwarded to MedGemma.
Only derived structured fields (typed dataclasses → JSON) are passed to the model.

---


if you see this that means you are on the latest commit!


In [1]:
print("if you see this that means you are on the latest commit!")


if you see this that means you are on the latest commit!


## Cell A: Setup & Imports

In [2]:
# Cell A-2: Library Installation
import subprocess, sys

packages = [
    "transformers>=4.40.0",
    "accelerate>=0.29.0",
    "sentencepiece>=0.1.99",
    "huggingface_hub>=0.22.0",
]

for pkg in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

print("Installation complete.")


Installation complete.


In [3]:
# Cell A-3: Core Imports
from __future__ import annotations
import re, json, warnings
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Optional, Tuple

try:
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM
    TRANSFORMERS_AVAILABLE = True
except ImportError:
    TRANSFORMERS_AVAILABLE = False
    print("transformers not available — stub mode will be used.")

print("Imports complete.")


Imports complete.


## Cell B: Data Models & Rule Library

In [4]:

from dataclasses import dataclass, field
from typing import List, Optional, Dict


@dataclass
class CultureReport:
    """
    Structured representation of a single culture lab report.

    Fields:
        date: ISO 8601 formatted date string (YYYY-MM-DD)
        organism: Name of identified organism (e.g., "E. coli")
        cfu: Colony Forming Units per mL
        resistance_markers: List of resistance markers (subset of ["ESBL","CRE","MRSA","VRE","CRKP"])
        specimen_type: Type of specimen ("urine" | "stool" | "unknown")
        contamination_flag: True if organism matches contamination terms
        raw_text: Original report string (NEVER passed to LLM)
    """

    date: str
    organism: str
    cfu: int
    resistance_markers: List[str]
    specimen_type: str
    contamination_flag: bool
    raw_text: str


@dataclass
class TrendResult:
    """
    Temporal comparison analysis across multiple culture reports.

    Fields:
        cfu_trend: "decreasing" | "increasing" | "fluctuating" | "cleared" | "insufficient_data"
        cfu_values: Ordered list of CFU values across reports
        cfu_deltas: Per-interval changes in CFU
        organism_persistent: True if same organism across all reports
        organism_list: Organism name per report
        resistance_evolution: True if new markers appear in later reports
        resistance_timeline: Resistance markers per report
        report_dates: ISO dates in sorted order
        any_contamination: True if any report flagged as contamination
    """

    cfu_trend: str
    cfu_values: List[int]
    cfu_deltas: List[int]
    organism_persistent: bool
    organism_list: List[str]
    resistance_evolution: bool
    resistance_timeline: List[List[str]]
    report_dates: List[str]
    any_contamination: bool


@dataclass
class HypothesisResult:
    """
    Rule-generated hypothesis with confidence scoring.

    Fields:
        interpretation: Natural language pattern summary (rule-generated)
        confidence: Confidence score [0.0, 0.95] - never 1.0
        risk_flags: List of risk flags (e.g., ["EMERGING_RESISTANCE", "CONTAMINATION"])
        stewardship_alert: True if resistance_evolution is True
        requires_clinician_review: Always True - structural safety guarantee
    """

    interpretation: str
    confidence: float
    risk_flags: List[str]
    stewardship_alert: bool
    requires_clinician_review: bool = True


@dataclass
class MedGemmaPayload:
    """
    Structured payload for MedGemma model inference.

    CRITICAL: raw_text from CultureReport is NEVER included in this payload.
    Only derived structured fields are forwarded.

    Fields:
        mode: "patient" | "clinician"
        trend_summary: Serialized TrendResult
        hypothesis_summary: Serialized HypothesisResult
        safety_constraints: Injected safety instructions
        output_schema: Expected output fields for this mode
    """

    mode: str
    trend_summary: dict
    hypothesis_summary: dict
    safety_constraints: List[str]
    output_schema: dict


@dataclass
class FormattedOutput:
    """
    Final rendered output for either Patient or Clinician mode.

    Fields are mode-specific. Patient mode uses patient_* fields,
    Clinician mode uses clinician_* fields.
    """

    mode: str

    # Patient mode fields
    patient_explanation: Optional[str] = None
    patient_trend_phrase: Optional[str] = None
    patient_questions: Optional[List[str]] = None
    patient_disclaimer: str = ""

    # Clinician mode fields
    clinician_trajectory: Optional[dict] = None
    clinician_interpretation: Optional[str] = None
    clinician_confidence: Optional[float] = None
    clinician_resistance_detail: Optional[str] = None
    clinician_stewardship_flag: Optional[bool] = None
    clinician_disclaimer: str = ""

In [5]:

# ---------------------------------------------------------------------------
# Core clinical rules and thresholds
# ---------------------------------------------------------------------------
RULES = {
    # CFU/mL threshold above which a urine specimen is considered infected
    "infection_threshold_urine": 100000,
    # CFU/mL threshold above which a stool specimen is considered infected
    "infection_threshold_stool": 50000,
    # A reduction of 75%+ from the previous reading is a strong improvement
    "significant_reduction_pct": 0.75,
    # Organism names indicating sample contamination rather than true infection
    "contamination_terms": [
        "mixed flora",
        "skin flora",
        "normal flora",
        "commensal",
        "contamination",
        "mixed growth",
    ],
    # High-risk resistance markers tracked by the rule engine
    "high_risk_markers": ["ESBL", "CRE", "MRSA", "VRE", "CRKP"],
    # CFU/mL at or below this value is treated as effectively cleared
    "cleared_threshold": 1000,
    # Hard ceiling on confidence - epistemic humility; never 1.0
    "max_confidence": 0.95,
    # Starting confidence before any signal adjustments
    "base_confidence": 0.50,
}

# ---------------------------------------------------------------------------
# Organism alias normalisation lookup table
# Maps common shorthand/abbreviations → canonical organism name.
# Matching is performed case-insensitively against stripped input.
# ---------------------------------------------------------------------------
ORGANISM_ALIASES: dict = {
    # Escherichia coli variants
    "e. coli": "Escherichia coli",
    "e.coli": "Escherichia coli",
    "e coli": "Escherichia coli",
    "escherichia coli": "Escherichia coli",
    # Klebsiella
    "klebsiella": "Klebsiella pneumoniae",
    "klebsiella pneumoniae": "Klebsiella pneumoniae",
    # Staphylococcus
    "staph aureus": "Staphylococcus aureus",
    "staphylococcus aureus": "Staphylococcus aureus",
    "s. aureus": "Staphylococcus aureus",
    "mrsa": "Staphylococcus aureus (MRSA)",
    # Enterococcus
    "enterococcus": "Enterococcus faecalis",
    "enterococcus faecalis": "Enterococcus faecalis",
    "e. faecalis": "Enterococcus faecalis",
    # Pseudomonas
    "pseudomonas": "Pseudomonas aeruginosa",
    "pseudomonas aeruginosa": "Pseudomonas aeruginosa",
    "p. aeruginosa": "Pseudomonas aeruginosa",
    # Proteus
    "proteus": "Proteus mirabilis",
    "proteus mirabilis": "Proteus mirabilis",
    # Contamination terms (kept as-is but included for normalisation completeness)
    "mixed flora": "mixed flora",
    "skin flora": "skin flora",
    "normal flora": "normal flora",
    "commensal": "commensal",
    "mixed growth": "mixed growth",
}


def normalize_organism(raw: str) -> str:
    """
    Normalise a raw organism string to its canonical name.

    Performs case-insensitive lookup against ORGANISM_ALIASES.
    Returns the canonical name if found, otherwise returns the stripped
    title-cased version of the original input.

    Args:
        raw: Raw organism string from extraction layer.

    Returns:
        Canonical organism name string.
    """
    key = raw.strip().lower()
    return ORGANISM_ALIASES.get(key, raw.strip())

## Cell C: Extraction Layer

In [6]:

import re
import warnings
import tempfile
from pathlib import Path
from typing import Optional



# ---------------------------------------------------------------------------
# Helper: Docling Processing
# ---------------------------------------------------------------------------
def _process_with_docling(input_text: str) -> str:
    """
    Process input text using Docling.

    If input_text is a valid file path, processes that file.
    Otherwise, writes text to a temporary file and processes it.
    Returns the structured markdown text from the document.
    """
    try:
        from docling.document_converter import DocumentConverter
    except ImportError:
        # Silently fail or log debug if needed, but for user-facing, return original text
        # Only warn once if desired, but here we just return
        return input_text

    input_path = Path(input_text)
    is_file = input_path.exists() and input_path.is_file()

    try:
        converter = DocumentConverter()

        if is_file:
            # Process directly from file path
            result = converter.convert(input_path)
            return result.document.export_to_markdown()
        else:
            # Input is raw text; Docling processing via temp file may distort layout (e.g. merging lines).
            # Fallback to returning raw text so regexes can use original newlines.
            return input_text

    except Exception as e:
        warnings.warn(
            f"Docling processing failed: {e}. Falling back to raw text.", UserWarning
        )
        return input_text


# ---------------------------------------------------------------------------
# Custom exception
# ---------------------------------------------------------------------------
class ExtractionError(ValueError):
    """Raised when both organism AND cfu fail to parse from a report."""


# ---------------------------------------------------------------------------
# Compiled regex patterns (Section 5.2)
# ---------------------------------------------------------------------------

# Organism: "Organism: <value>" up to newline or end of string
_RE_ORGANISM = re.compile(r"Organism:\s*(.+?)(?:\n|$)", re.IGNORECASE)

# CFU/mL: "CFU/mL: <digits with optional commas>"
_RE_CFU_PRIMARY = re.compile(r"CFU/mL:\s*([\d,]+)", re.IGNORECASE)

# Fallback CFU patterns
_RE_CFU_SCIENTIFIC = re.compile(r"10\^(\d+)", re.IGNORECASE)  # 10^5 → 100000
_RE_CFU_WORD = re.compile(r"(TNTC|Too\s+Numerous\s+To\s+Count)", re.IGNORECASE)
_RE_CFU_NO_GROWTH = re.compile(r"(No\s+growth|0\s+CFU)", re.IGNORECASE)
_RE_CFU_RAW_NUMBER = re.compile(r"\b([\d]{4,})\b")  # bare large number

# Date: ISO 8601 or MM/DD/YYYY
_RE_DATE_PRIMARY = re.compile(
    r"(?:Date|Collected|Reported):\s*(\d{4}-\d{2}-\d{2}|\d{2}/\d{2}/\d{4})",
    re.IGNORECASE,
)
_RE_DATE_FALLBACK = re.compile(r"\b(\d{4}-\d{2}-\d{2})\b")

# Resistance markers: exact case-insensitive word boundaries
_RE_RESISTANCE = re.compile(r"\b(ESBL|CRE|MRSA|VRE|CRKP)\b", re.IGNORECASE)

# Specimen type
_RE_SPECIMEN = re.compile(
    r"Specimen:\s*(urine|stool|wound|blood)", re.IGNORECASE)


# ---------------------------------------------------------------------------
# CFU normalisation helper (Section 5.4)
# ---------------------------------------------------------------------------


def _parse_cfu(report_text: str) -> tuple[int, bool]:
    """
    Attempt to parse the CFU/mL value from a report text string.

    Returns:
        (cfu_value, parse_success) tuple.

    Normalisation rules:
        - "TNTC" / "Too Numerous To Count" → 999999
        - "No growth" / "0 CFU"            → 0
        - "10^5"                            → 100000
        - comma-separated integer           → int (commas stripped)
        - Missing/unparseable               → 0 with warning
    """
    # 1. Primary: "CFU/mL: 120,000"
    m = _RE_CFU_PRIMARY.search(report_text)
    if m:
        raw = m.group(1).replace(",", "")
        try:
            return int(raw), True
        except ValueError:
            pass

    # 2. TNTC
    if _RE_CFU_WORD.search(report_text):
        return 999999, True

    # 3. No growth
    if _RE_CFU_NO_GROWTH.search(report_text):
        return 0, True

    # 4. Scientific notation "10^5"
    m = _RE_CFU_SCIENTIFIC.search(report_text)
    if m:
        try:
            return 10 ** int(m.group(1)), True
        except (ValueError, OverflowError):
            pass

    # 5. Bare large integer (≥4 digits) — last resort fallback
    m = _RE_CFU_RAW_NUMBER.search(report_text)
    if m:
        raw = m.group(1).replace(",", "")
        try:
            val = int(raw)
            warnings.warn(
                f"CFU parsed from bare number '{raw}' — review report text.",
                UserWarning,
                stacklevel=3,
            )
            return val, True
        except ValueError:
            pass

    warnings.warn(
        "CFU/mL could not be parsed; defaulting to 0.", UserWarning, stacklevel=3
    )
    return 0, False


def _parse_date(report_text: str) -> str:
    """Extract and normalise the collection date from report text."""
    m = _RE_DATE_PRIMARY.search(report_text)
    if m:
        raw = m.group(1)
        # Convert MM/DD/YYYY → YYYY-MM-DD
        if "/" in raw:
            parts = raw.split("/")
            return f"{parts[2]}-{parts[0].zfill(2)}-{parts[1].zfill(2)}"
        return raw

    # Fallback: any ISO date in text
    m = _RE_DATE_FALLBACK.search(report_text)
    if m:
        return m.group(1)

    return "unknown"


def _parse_organism(report_text: str) -> Optional[str]:
    """
    Extract organism name from report text.

    Primary: "Organism: <value>"
    Fallback: scan for known organism names / aliases.
    """
    m = _RE_ORGANISM.search(report_text)
    if m:
        return normalize_organism(m.group(1).strip())

    # Fallback: search for known organism aliases in full text
    lower_text = report_text.lower()

    for alias in sorted(ORGANISM_ALIASES.keys(), key=len, reverse=True):
        if alias in lower_text:
            return normalize_organism(alias)

    return None


def _parse_resistance_markers(report_text: str) -> list[str]:
    """Extract all high-risk resistance markers (deduplicated, uppercase)."""
    found = _RE_RESISTANCE.findall(report_text)
    # deduplicate, preserve order
    return list(dict.fromkeys(m.upper() for m in found))


def _parse_specimen(report_text: str) -> str:
    """Extract specimen type; defaults to 'unknown'."""
    m = _RE_SPECIMEN.search(report_text)
    if m:
        return m.group(1).lower()
    return "unknown"


def _is_contamination(organism: str) -> bool:
    """Return True if the organism name matches any contamination term."""
    lower = organism.lower()
    return any(term in lower for term in RULES["contamination_terms"])


# ---------------------------------------------------------------------------
# Public extraction function
# ---------------------------------------------------------------------------


def extract_structured_data(report_text: str) -> CultureReport:
    """
    Parse a free-text culture report into a typed CultureReport.

    Now supports direct file paths via Docling processing.

    Rules:
        - Organism field: stripped, normalised via ORGANISM_ALIASES
        - CFU: commas removed, converted to int; TNTC=999999
        - resistance_markers: deduplicated, uppercase
        - contamination_flag: True if organism in contamination_terms
        - raw_text: stored as-is (or docling processed), NEVER forwarded to MedGemma

    Raises:
        ExtractionError: if both organism AND cfu fail to parse.
    """
    # Pre-process with Docling (handles file paths or raw text)
    processed_text = _process_with_docling(report_text)

    # Attempt extraction on processed text
    organism = _parse_organism(processed_text)
    cfu, cfu_ok = _parse_cfu(processed_text)

    # Fallback: if extraction failed and text was modified by Docling, try original
    if (organism is None and not cfu_ok) and processed_text != report_text:
        organism = _parse_organism(report_text)
        cfu, cfu_ok = _parse_cfu(report_text)
        if organism is not None or cfu_ok:
            processed_text = report_text  # Revert to original for other fields

    if organism is None and not cfu_ok:
        raise ExtractionError(
            "Extraction failed: could not parse organism OR CFU/mL from report. "
            "Check report format."
        )

    # If only organism failed, use a placeholder and warn
    if organism is None:
        warnings.warn(
            "Organism could not be parsed; using 'unknown'.", UserWarning, stacklevel=2
        )
        organism = "unknown"

    resistance_markers = _parse_resistance_markers(processed_text)
    specimen_type = _parse_specimen(processed_text)
    contamination_flag = _is_contamination(organism)
    date = _parse_date(processed_text)

    return CultureReport(
        date=date,
        organism=organism,
        cfu=cfu,
        resistance_markers=resistance_markers,
        specimen_type=specimen_type,
        contamination_flag=contamination_flag,
        raw_text=processed_text,  # Store the text actually used for extraction
    )

In [7]:
# --- Extraction Unit Tests ---

import warnings

_PASS = 0
_FAIL = 0


def _assert(condition: bool, msg: str) -> None:
    global _PASS, _FAIL
    if condition:
        _PASS += 1
        print(f"  PASS  {msg}")
    else:
        _FAIL += 1
        print(f"  FAIL  {msg}")


# ---------------------------------------------------------------------------
# Test Report 1 — Normal improving report
# ---------------------------------------------------------------------------
REPORT_NORMAL = """
Specimen: Urine
Date Collected: 2026-01-01
Organism: E. coli
CFU/mL: 120,000
Sensitivity: Ampicillin - Resistant, Nitrofurantoin - Sensitive
"""

print("=== Test: Normal Report ===")
r = extract_structured_data(REPORT_NORMAL)
_assert(r.date == "2026-01-01", f"date == '2026-01-01'  (got '{r.date}')")
_assert(
    r.organism == "Escherichia coli",
    f"organism normalised to 'Escherichia coli'  (got '{r.organism}')",
)
_assert(r.cfu == 120000, f"cfu == 120000  (got {r.cfu})")
_assert(
    r.resistance_markers == [], f"no resistance markers  (got {r.resistance_markers})"
)
_assert(
    r.specimen_type == "urine", f"specimen_type == 'urine'  (got '{r.specimen_type}')"
)
_assert(
    r.contamination_flag is False,
    f"contamination_flag is False  (got {r.contamination_flag})",
)

# ---------------------------------------------------------------------------
# Test Report 2 — Contamination report (mixed flora, low CFU)
# ---------------------------------------------------------------------------
REPORT_CONTAMINATION = """
Specimen: Urine
Date Collected: 2026-02-05
Organism: mixed flora
CFU/mL: 5,000
No resistance markers detected.
"""

print("\n=== Test: Contamination Report ===")
r2 = extract_structured_data(REPORT_CONTAMINATION)
_assert(
    r2.contamination_flag is True,
    f"contamination_flag is True  (got {r2.contamination_flag})",
)
_assert(
    r2.organism == "mixed flora", f"organism == 'mixed flora'  (got '{r2.organism}')"
)
_assert(r2.cfu == 5000, f"cfu == 5000  (got {r2.cfu})")
_assert(
    r2.resistance_markers == [], f"no resistance markers  (got {r2.resistance_markers})"
)

# ---------------------------------------------------------------------------
# Test Report 3 — Resistance-containing report (ESBL marker)
# ---------------------------------------------------------------------------
REPORT_RESISTANCE = """
Specimen: Urine
Date Collected: 2026-01-20
Organism: Klebsiella pneumoniae
CFU/mL: 75,000
Resistance: ESBL detected.
"""

print("\n=== Test: Resistance Report ===")
r3 = extract_structured_data(REPORT_RESISTANCE)
_assert(
    r3.organism == "Klebsiella pneumoniae",
    f"organism == 'Klebsiella pneumoniae'  (got '{r3.organism}')",
)
_assert(
    "ESBL" in r3.resistance_markers,
    f"ESBL in resistance_markers  (got {r3.resistance_markers})",
)
_assert(
    r3.contamination_flag is False,
    f"contamination_flag is False  (got {r3.contamination_flag})",
)
_assert(r3.cfu == 75000, f"cfu == 75000  (got {r3.cfu})")

# ---------------------------------------------------------------------------
# Test — TNTC CFU normalisation
# ---------------------------------------------------------------------------
REPORT_TNTC = """
Specimen: Urine
Date Collected: 2026-03-01
Organism: E. coli
CFU/mL: TNTC
"""

print("\n=== Test: TNTC Normalisation ===")
r4 = extract_structured_data(REPORT_TNTC)
_assert(r4.cfu == 999999, f"TNTC → 999999  (got {r4.cfu})")

# ---------------------------------------------------------------------------
# Test — No growth / cleared
# ---------------------------------------------------------------------------
REPORT_NO_GROWTH = """
Specimen: Urine
Date Collected: 2026-03-15
Organism: E. coli
No growth observed.
"""

print("\n=== Test: No Growth ===")
r5 = extract_structured_data(REPORT_NO_GROWTH)
_assert(r5.cfu == 0, f"No growth → cfu == 0  (got {r5.cfu})")

# ---------------------------------------------------------------------------
# Test — ExtractionError on completely unparseable input
# ---------------------------------------------------------------------------
print("\n=== Test: ExtractionError on bad input ===")
try:
    extract_structured_data("this report contains absolutely nothing useful at all")
    _assert(False, "ExtractionError should have been raised")
except ExtractionError as e:
    _assert(True, f"ExtractionError raised correctly: {e}")
except Exception as e:
    _assert(False, f"Wrong exception type raised: {type(e).__name__}: {e}")

# ---------------------------------------------------------------------------
# Test — Adversarial: SQL injection in CFU field
# ---------------------------------------------------------------------------
REPORT_ADV = """
Specimen: Urine
Date Collected: 2026-04-01
Organism: E. coli
CFU/mL: 100000; DROP TABLE reports
"""

print("\n=== Test: Adversarial SQL Injection in CFU ===")
# Should parse 100000 from the start, or fallback gracefully
try:
    r6 = extract_structured_data(REPORT_ADV)
    # The regex only captures digits+commas, so "100000" is parsed, the rest is ignored
    _assert(r6.cfu == 100000, f"cfu == 100000 (injection ignored)  (got {r6.cfu})")
except ExtractionError:
    _assert(False, "Should not raise ExtractionError on adversarial CFU")

# ---------------------------------------------------------------------------
# Test — Alternate date format MM/DD/YYYY
# ---------------------------------------------------------------------------
REPORT_DATE_ALT = """
Specimen: Stool
Date Collected: 01/15/2026
Organism: Enterococcus faecalis
CFU/mL: 60,000
"""

print("\n=== Test: Alternate Date Format (MM/DD/YYYY) ===")
r7 = extract_structured_data(REPORT_DATE_ALT)
_assert(r7.date == "2026-01-15", f"date normalised to ISO  (got '{r7.date}')")
_assert(
    r7.specimen_type == "stool", f"specimen_type == 'stool'  (got '{r7.specimen_type}')"
)

# ---------------------------------------------------------------------------
# Summary
# ---------------------------------------------------------------------------
print(f"\n{'=' * 50}")
print(f"Extraction Tests Complete: {_PASS} passed, {_FAIL} failed")
if _FAIL == 0:
    print("ALL TESTS PASSED")
else:
    print(f"WARNING: {_FAIL} test(s) failed — review extraction logic")

=== Test: Normal Report ===
  PASS  date == '2026-01-01'  (got '2026-01-01')
  PASS  organism normalised to 'Escherichia coli'  (got 'Escherichia coli')
  PASS  cfu == 120000  (got 120000)
  PASS  no resistance markers  (got [])
  PASS  specimen_type == 'urine'  (got 'urine')
  PASS  contamination_flag is False  (got False)

=== Test: Contamination Report ===
  PASS  contamination_flag is True  (got True)
  PASS  organism == 'mixed flora'  (got 'mixed flora')
  PASS  cfu == 5000  (got 5000)
  PASS  no resistance markers  (got [])

=== Test: Resistance Report ===
  PASS  organism == 'Klebsiella pneumoniae'  (got 'Klebsiella pneumoniae')
  PASS  ESBL in resistance_markers  (got ['ESBL'])
  PASS  contamination_flag is False  (got False)
  PASS  cfu == 75000  (got 75000)

=== Test: TNTC Normalisation ===
  PASS  TNTC → 999999  (got 999999)

=== Test: No Growth ===
  PASS  No growth → cfu == 0  (got 0)

=== Test: ExtractionError on bad input ===
  PASS  ExtractionError raised correctly: Ext

/tmp/ipython-input-1549518826.py:134: UserWarning: CFU/mL could not be parsed; defaulting to 0.
  extract_structured_data("this report contains absolutely nothing useful at all")


## Cell D: Temporal Trend Engine

In [8]:

from typing import List



# ---------------------------------------------------------------------------
# Internal helpers
# ---------------------------------------------------------------------------


def _classify_cfu_trend(cfu_values: List[int]) -> str:
    """
    Classify the CFU trajectory from an ordered list of values.

    Labels (priority order):
        "insufficient_data"  — fewer than 2 reports
        "cleared"            — final value ≤ cleared_threshold (overrides all)
        "decreasing"         — all values monotonically decreasing
        "increasing"         — all values monotonically increasing
        "fluctuating"        — any other pattern
    """
    if len(cfu_values) < 2:
        return "insufficient_data"

    # "cleared" overrides all other labels
    if cfu_values[-1] <= RULES["cleared_threshold"]:
        return "cleared"

    strictly_decreasing = all(
        cfu_values[i] > cfu_values[i + 1] for i in range(len(cfu_values) - 1)
    )
    if strictly_decreasing:
        return "decreasing"

    strictly_increasing = all(
        cfu_values[i] < cfu_values[i + 1] for i in range(len(cfu_values) - 1)
    )
    if strictly_increasing:
        return "increasing"

    return "fluctuating"


def _compute_deltas(cfu_values: List[int]) -> List[int]:
    """
    Compute per-interval CFU changes.

    Positive delta = worsening (increasing CFU).
    Negative delta = improving (decreasing CFU).
    """
    return [cfu_values[i + 1] - cfu_values[i] for i in range(len(cfu_values) - 1)]


def _check_persistence(organism_list: List[str]) -> bool:
    """
    Return True if the same organism was isolated across all reports.

    Comparison is performed on normalised (lowercase, stripped) organism names,
    with alias resolution to handle "E. coli" == "Escherichia coli".
    """
    normalised = [normalize_organism(o).strip().lower() for o in organism_list]
    return len(set(normalised)) == 1


def _check_resistance_evolution(reports: List[CultureReport]) -> bool:
    """
    Return True if new resistance markers appear in any report after the first.

    Logic:
        - Baseline = markers in report[0]
        - If any subsequent report contains a marker not in baseline → True
    """
    if len(reports) < 2:
        return False
    baseline = set(reports[0].resistance_markers)
    later_markers: set[str] = set()
    for r in reports[1:]:
        later_markers.update(r.resistance_markers)
    return bool(later_markers - baseline)


def _build_resistance_timeline(reports: List[CultureReport]) -> List[List[str]]:
    """Return per-report resistance marker lists, in report order."""
    return [list(r.resistance_markers) for r in reports]


# ---------------------------------------------------------------------------
# Public API
# ---------------------------------------------------------------------------


def analyze_trend(reports: List[CultureReport]) -> TrendResult:
    """
    Compute a TrendResult from an ordered list of CultureReport objects.

    Reports should be sorted by date (oldest first) before calling this
    function. The function does NOT re-sort — caller is responsible.

    Args:
        reports: 1–3 CultureReport instances in chronological order.

    Returns:
        TrendResult with all temporal signal fields populated.
    """
    if not reports:
        raise ValueError("analyze_trend requires at least one CultureReport.")

    cfu_values = [r.cfu for r in reports]
    cfu_deltas = _compute_deltas(cfu_values)
    cfu_trend = _classify_cfu_trend(cfu_values)
    organism_list = [r.organism for r in reports]
    organism_persistent = _check_persistence(organism_list)
    resistance_evolution = _check_resistance_evolution(reports)
    resistance_timeline = _build_resistance_timeline(reports)
    report_dates = [r.date for r in reports]
    any_contamination = any(r.contamination_flag for r in reports)

    return TrendResult(
        cfu_trend=cfu_trend,
        cfu_values=cfu_values,
        cfu_deltas=cfu_deltas,
        organism_persistent=organism_persistent,
        organism_list=organism_list,
        resistance_evolution=resistance_evolution,
        resistance_timeline=resistance_timeline,
        report_dates=report_dates,
        any_contamination=any_contamination,
    )

In [9]:
# --- Trend Unit Tests ---


_PASS = 0
_FAIL = 0


def _assert(condition: bool, msg: str) -> None:
    global _PASS, _FAIL
    if condition:
        _PASS += 1
        print(f"  PASS  {msg}")
    else:
        _FAIL += 1
        print(f"  FAIL  {msg}")


def _make_report(
    cfu: int,
    organism: str = "Escherichia coli",
    date: str = "2026-01-01",
    markers=None,
    contamination: bool = False,
) -> CultureReport:
    return CultureReport(
        date=date,
        organism=organism,
        cfu=cfu,
        resistance_markers=markers or [],
        specimen_type="urine",
        contamination_flag=contamination,
        raw_text="<stub>",
    )


# ---------------------------------------------------------------------------
# 1. Monotonically decreasing
# ---------------------------------------------------------------------------
print("=== Test: Monotonically Decreasing CFU ===")
rpts = [
    _make_report(120000, date="2026-01-01"),
    _make_report(40000, date="2026-01-10"),
    _make_report(5000, date="2026-01-20"),
]
t = analyze_trend(rpts)
_assert(t.cfu_trend == "decreasing", f"trend == 'decreasing'  (got '{t.cfu_trend}')")
_assert(t.cfu_deltas == [-80000, -35000], f"deltas correct  (got {t.cfu_deltas})")
_assert(t.organism_persistent is True, f"organism_persistent == True")
_assert(t.resistance_evolution is False, f"resistance_evolution == False")
_assert(t.any_contamination is False, f"any_contamination == False")

# ---------------------------------------------------------------------------
# 2. Cleared (final CFU ≤ 1000) — overrides decreasing
# ---------------------------------------------------------------------------
print("\n=== Test: Cleared (Final CFU ≤ 1000) ===")
rpts2 = [
    _make_report(120000, date="2026-01-01"),
    _make_report(40000, date="2026-01-10"),
    _make_report(800, date="2026-01-20"),
]
t2 = analyze_trend(rpts2)
_assert(t2.cfu_trend == "cleared", f"trend == 'cleared'  (got '{t2.cfu_trend}')")

# ---------------------------------------------------------------------------
# 3. CFU = 0 (no growth) → also cleared
# ---------------------------------------------------------------------------
print("\n=== Test: Zero CFU (No Growth) ===")
rpts3 = [
    _make_report(80000, date="2026-01-01"),
    _make_report(0, date="2026-01-10"),
]
t3 = analyze_trend(rpts3)
_assert(
    t3.cfu_trend == "cleared", f"trend == 'cleared' for CFU=0  (got '{t3.cfu_trend}')"
)

# ---------------------------------------------------------------------------
# 4. Monotonically increasing
# ---------------------------------------------------------------------------
print("\n=== Test: Monotonically Increasing CFU ===")
rpts4 = [
    _make_report(40000, date="2026-01-01"),
    _make_report(80000, date="2026-01-10"),
    _make_report(120000, date="2026-01-20"),
]
t4 = analyze_trend(rpts4)
_assert(t4.cfu_trend == "increasing", f"trend == 'increasing'  (got '{t4.cfu_trend}')")

# ---------------------------------------------------------------------------
# 5. Fluctuating
# ---------------------------------------------------------------------------
print("\n=== Test: Fluctuating CFU ===")
rpts5 = [
    _make_report(80000, date="2026-01-01"),
    _make_report(120000, date="2026-01-10"),
    _make_report(60000, date="2026-01-20"),
]
t5 = analyze_trend(rpts5)
_assert(
    t5.cfu_trend == "fluctuating", f"trend == 'fluctuating'  (got '{t5.cfu_trend}')"
)

# ---------------------------------------------------------------------------
# 6. Single report — insufficient_data
# ---------------------------------------------------------------------------
print("\n=== Test: Single Report (Insufficient Data) ===")
rpts6 = [_make_report(100000, date="2026-01-01")]
t6 = analyze_trend(rpts6)
_assert(
    t6.cfu_trend == "insufficient_data",
    f"trend == 'insufficient_data'  (got '{t6.cfu_trend}')",
)
_assert(t6.cfu_deltas == [], f"deltas == []  (got {t6.cfu_deltas})")

# ---------------------------------------------------------------------------
# 7. Resistance evolution detection
# ---------------------------------------------------------------------------
print("\n=== Test: Resistance Evolution ===")
rpts7 = [
    _make_report(90000, date="2026-01-01", markers=[]),
    _make_report(80000, date="2026-01-10", markers=[]),
    _make_report(75000, date="2026-01-20", markers=["ESBL"]),
]
t7 = analyze_trend(rpts7)
_assert(t7.resistance_evolution is True, f"resistance_evolution == True")
_assert(t7.resistance_timeline[2] == ["ESBL"], f"resistance_timeline[2] == ['ESBL']")

# ---------------------------------------------------------------------------
# 8. Organism change (not persistent)
# ---------------------------------------------------------------------------
print("\n=== Test: Organism Change ===")
rpts8 = [
    _make_report(100000, organism="Escherichia coli", date="2026-01-01"),
    _make_report(90000, organism="Klebsiella pneumoniae", date="2026-01-10"),
]
t8 = analyze_trend(rpts8)
_assert(
    t8.organism_persistent is False,
    f"organism_persistent == False when organism changes",
)

# ---------------------------------------------------------------------------
# 9. Contamination flag propagation
# ---------------------------------------------------------------------------
print("\n=== Test: Contamination Propagation ===")
rpts9 = [
    _make_report(5000, organism="mixed flora", date="2026-01-01", contamination=True),
    _make_report(3000, organism="mixed flora", date="2026-01-10", contamination=True),
]
t9 = analyze_trend(rpts9)
_assert(t9.any_contamination is True, f"any_contamination == True")

# ---------------------------------------------------------------------------
# Summary
# ---------------------------------------------------------------------------
print(f"\n{'=' * 50}")
print(f"Trend Tests Complete: {_PASS} passed, {_FAIL} failed")
if _FAIL == 0:
    print("ALL TESTS PASSED")
else:
    print(f"WARNING: {_FAIL} test(s) failed")

=== Test: Monotonically Decreasing CFU ===
  PASS  trend == 'decreasing'  (got 'decreasing')
  PASS  deltas correct  (got [-80000, -35000])
  PASS  organism_persistent == True
  PASS  resistance_evolution == False
  PASS  any_contamination == False

=== Test: Cleared (Final CFU ≤ 1000) ===
  PASS  trend == 'cleared'  (got 'cleared')

=== Test: Zero CFU (No Growth) ===
  PASS  trend == 'cleared' for CFU=0  (got 'cleared')

=== Test: Monotonically Increasing CFU ===
  PASS  trend == 'increasing'  (got 'increasing')

=== Test: Fluctuating CFU ===
  PASS  trend == 'fluctuating'  (got 'fluctuating')

=== Test: Single Report (Insufficient Data) ===
  PASS  trend == 'insufficient_data'  (got 'insufficient_data')
  PASS  deltas == []  (got [])

=== Test: Resistance Evolution ===
  PASS  resistance_evolution == True
  PASS  resistance_timeline[2] == ['ESBL']

=== Test: Organism Change ===
  PASS  organism_persistent == False when organism changes

=== Test: Contamination Propagation ===
  PASS 

## Cell E: Hypothesis Update Layer

In [10]:

from typing import List



# ---------------------------------------------------------------------------
# Risk flag constants
# ---------------------------------------------------------------------------
FLAG_EMERGING_RESISTANCE = "EMERGING_RESISTANCE"
FLAG_CONTAMINATION = "CONTAMINATION_SUSPECTED"
FLAG_NON_RESPONSE = "NON_RESPONSE_PATTERN"
FLAG_INSUFFICIENT_DATA = "INSUFFICIENT_DATA"
FLAG_ORGANISM_CHANGE = "ORGANISM_CHANGE"


# ---------------------------------------------------------------------------
# Confidence scoring
# ---------------------------------------------------------------------------


def _score_confidence(trend: TrendResult, report_count: int) -> float:
    """
    Apply deterministic signal adjustments to a base confidence value.

    Starting point: RULES["base_confidence"] = 0.50
    Each signal adds or subtracts a fixed delta.
    Final value is clamped to [0.0, RULES["max_confidence"]].

    Signal table (Section 7.1):
        +0.30  CFU decreasing
        +0.40  CFU cleared
        +0.20  CFU increasing  (high confidence of non-response)
        -0.10  CFU fluctuating
        -0.10  resistance evolution
        -0.05  organism changed
        -0.20  contamination present
        -0.25  fewer than 2 reports
    """
    confidence = RULES["base_confidence"]

    # Trend signal
    if trend.cfu_trend == "decreasing":
        confidence += 0.30
    elif trend.cfu_trend == "cleared":
        confidence += 0.40
    elif trend.cfu_trend == "increasing":
        confidence += 0.20  # high confidence of non-response
    elif trend.cfu_trend == "fluctuating":
        confidence -= 0.10

    # Resistance evolution penalty
    if trend.resistance_evolution:
        confidence -= 0.10

    # Organism change uncertainty
    if not trend.organism_persistent:
        confidence -= 0.05

    # Contamination validity concern
    if trend.any_contamination:
        confidence -= 0.20

    # Insufficient data penalty
    if report_count < 2:
        confidence -= 0.25

    # Hard clamp: never < 0.0, never > max_confidence (epistemic humility)
    return round(max(0.0, min(confidence, RULES["max_confidence"])), 4)


# ---------------------------------------------------------------------------
# Risk flag assignment (Section 7.2)
# ---------------------------------------------------------------------------


def _assign_risk_flags(trend: TrendResult, report_count: int) -> List[str]:
    """Build a list of risk flag strings from trend signals."""
    flags: List[str] = []

    if trend.resistance_evolution:
        flags.append(FLAG_EMERGING_RESISTANCE)

    if trend.any_contamination:
        flags.append(FLAG_CONTAMINATION)

    if trend.cfu_trend == "increasing":
        flags.append(FLAG_NON_RESPONSE)

    if report_count < 2:
        flags.append(FLAG_INSUFFICIENT_DATA)

    if not trend.organism_persistent:
        flags.append(FLAG_ORGANISM_CHANGE)

    return flags


# ---------------------------------------------------------------------------
# Interpretation string construction (Section 7.3)
# ---------------------------------------------------------------------------


def _build_interpretation(trend: TrendResult, report_count: int) -> str:
    """
    Construct a rule-generated natural language pattern summary.

    This string is passed to MedGemma only as structured context inside
    the JSON payload — never as a direct LLM prompt.
    """
    parts: List[str] = []

    if trend.cfu_trend == "decreasing":
        parts.append("Pattern suggests improving infection response.")
    elif trend.cfu_trend == "cleared":
        parts.append("Pattern suggests possible resolution.")
    elif trend.cfu_trend == "increasing":
        parts.append("Pattern suggests possible non-response.")
    elif trend.cfu_trend == "fluctuating":
        parts.append("Pattern is variable — requires clinical context.")
    elif trend.cfu_trend == "insufficient_data":
        parts.append("Insufficient longitudinal data for trend analysis.")

    if trend.resistance_evolution:
        parts.append("Emerging resistance observed.")

    if not trend.organism_persistent:
        parts.append("Organism change may indicate reinfection.")

    if trend.any_contamination:
        parts.append("Contamination suspected — interpret with caution.")

    return " ".join(parts)


# ---------------------------------------------------------------------------
# Public API
# ---------------------------------------------------------------------------


def generate_hypothesis(trend: TrendResult, report_count: int) -> HypothesisResult:
    """
    Generate a deterministic hypothesis from a TrendResult.

    Args:
        trend: Computed TrendResult from the trend engine.
        report_count: Number of source reports (used for insufficient-data logic).

    Returns:
        HypothesisResult with confidence score, risk flags, interpretation,
        stewardship alert, and mandatory clinician review flag.
    """
    confidence = _score_confidence(trend, report_count)
    risk_flags = _assign_risk_flags(trend, report_count)
    interpretation = _build_interpretation(trend, report_count)
    stewardship_alert = trend.resistance_evolution

    return HypothesisResult(
        interpretation=interpretation,
        confidence=confidence,
        risk_flags=risk_flags,
        stewardship_alert=stewardship_alert,
        requires_clinician_review=True,  # Always True — structural safety guarantee
    )

In [11]:
# --- Hypothesis Unit Tests ---


_PASS = 0
_FAIL = 0


def _assert(condition: bool, msg: str) -> None:
    global _PASS, _FAIL
    if condition:
        _PASS += 1
        print(f"  PASS  {msg}")
    else:
        _FAIL += 1
        print(f"  FAIL  {msg}")


def _make_report(
    cfu: int,
    organism: str = "Escherichia coli",
    date: str = "2026-01-01",
    markers=None,
    contamination: bool = False,
) -> CultureReport:
    return CultureReport(
        date=date,
        organism=organism,
        cfu=cfu,
        resistance_markers=markers or [],
        specimen_type="urine",
        contamination_flag=contamination,
        raw_text="<stub>",
    )


# ---------------------------------------------------------------------------
# 1. Perfect improvement (decreasing → cleared) — confidence ≥ 0.80
# ---------------------------------------------------------------------------
print("=== Test: Perfect Improvement (Decreasing → Cleared) ===")
rpts = [
    _make_report(120000, date="2026-01-01"),
    _make_report(40000, date="2026-01-10"),
    _make_report(800, date="2026-01-20"),  # cleared (≤ 1000)
]
trend = analyze_trend(rpts)
hyp = generate_hypothesis(trend, len(rpts))

_assert(
    hyp.confidence >= 0.80,
    f"confidence ≥ 0.80 for cleared trend  (got {hyp.confidence})",
)
_assert(
    hyp.confidence <= 0.95, f"confidence ≤ 0.95 (hard ceiling)  (got {hyp.confidence})"
)
_assert(hyp.stewardship_alert is False, f"stewardship_alert == False")
_assert(hyp.requires_clinician_review is True, f"requires_clinician_review always True")
_assert(
    "possible resolution" in hyp.interpretation, f"interpretation mentions resolution"
)

# ---------------------------------------------------------------------------
# 2. Emerging resistance — confidence drops vs. clean improving scenario
# ---------------------------------------------------------------------------
print("\n=== Test: Emerging Resistance (Confidence Drops) ===")
rpts2 = [
    _make_report(90000, date="2026-01-01", markers=[]),
    _make_report(80000, date="2026-01-10", markers=[]),
    _make_report(75000, date="2026-01-20", markers=["ESBL"]),
]
trend2 = analyze_trend(rpts2)
hyp2 = generate_hypothesis(trend2, len(rpts2))

_assert(
    FLAG_EMERGING_RESISTANCE in hyp2.risk_flags, f"EMERGING_RESISTANCE in risk_flags"
)
_assert(hyp2.stewardship_alert is True, f"stewardship_alert == True")
_assert(
    hyp2.confidence < 0.80,
    f"confidence < 0.80 when resistance emerges  (got {hyp2.confidence})",
)

# ---------------------------------------------------------------------------
# 3. Contamination — confidence is reduced by the -0.20 contamination penalty.
#    With decreasing CFU (5000→3000): base 0.50 + 0.30 (decreasing) - 0.20 (contamination) = 0.60
#    The PRD Appendix B example uses a fluctuating pattern; here decreasing gives 0.60.
# ---------------------------------------------------------------------------
print("\n=== Test: Contamination (Confidence Drops Sharply) ===")
rpts3 = [
    _make_report(5000, organism="mixed flora", date="2026-01-01", contamination=True),
    _make_report(3000, organism="mixed flora", date="2026-01-10", contamination=True),
]
trend3 = analyze_trend(rpts3)
hyp3 = generate_hypothesis(trend3, len(rpts3))

_assert(FLAG_CONTAMINATION in hyp3.risk_flags, f"CONTAMINATION_SUSPECTED in risk_flags")
_assert(
    hyp3.confidence <= 0.65,
    f"confidence reduced by contamination penalty (got {hyp3.confidence})",
)
_assert(
    "Contamination suspected" in hyp3.interpretation,
    f"interpretation flags contamination",
)

# ---------------------------------------------------------------------------
# 4. Single report — insufficient data penalty
# ---------------------------------------------------------------------------
print("\n=== Test: Single Report (Insufficient Data) ===")
rpts4 = [_make_report(100000, date="2026-01-01")]
trend4 = analyze_trend(rpts4)
hyp4 = generate_hypothesis(trend4, len(rpts4))

_assert(
    hyp4.confidence == 0.25,
    f"confidence == 0.25 (base 0.50 - 0.25)  (got {hyp4.confidence})",
)
_assert("INSUFFICIENT_DATA" in hyp4.risk_flags, f"INSUFFICIENT_DATA in risk_flags")

# ---------------------------------------------------------------------------
# 5. Increasing CFU — non-response pattern
# ---------------------------------------------------------------------------
print("\n=== Test: Increasing CFU (Non-Response) ===")
rpts5 = [
    _make_report(40000, date="2026-01-01"),
    _make_report(80000, date="2026-01-10"),
    _make_report(120000, date="2026-01-20"),
]
trend5 = analyze_trend(rpts5)
hyp5 = generate_hypothesis(trend5, len(rpts5))

_assert(
    "NON_RESPONSE_PATTERN" in hyp5.risk_flags, f"NON_RESPONSE_PATTERN in risk_flags"
)
_assert(
    hyp5.confidence == 0.70,
    f"confidence == 0.70 (0.50 + 0.20)  (got {hyp5.confidence})",
)
_assert(
    "non-response" in hyp5.interpretation.lower(),
    f"interpretation mentions non-response",
)

# ---------------------------------------------------------------------------
# 6. Confidence never exceeds 0.95
# ---------------------------------------------------------------------------
print("\n=== Test: Confidence Hard Ceiling ===")
# Best possible scenario: cleared, persistent, no resistance, no contamination
rpts6 = [
    _make_report(120000, date="2026-01-01"),
    _make_report(800, date="2026-01-10"),  # cleared
]
trend6 = analyze_trend(rpts6)
hyp6 = generate_hypothesis(trend6, len(rpts6))
_assert(
    hyp6.confidence <= 0.95, f"confidence never exceeds 0.95  (got {hyp6.confidence})"
)

# ---------------------------------------------------------------------------
# Summary
# ---------------------------------------------------------------------------
print(f"\n{'=' * 50}")
print(f"Hypothesis Tests Complete: {_PASS} passed, {_FAIL} failed")
if _FAIL == 0:
    print("ALL TESTS PASSED")
else:
    print(f"WARNING: {_FAIL} test(s) failed")

=== Test: Perfect Improvement (Decreasing → Cleared) ===
  PASS  confidence ≥ 0.80 for cleared trend  (got 0.9)
  PASS  confidence ≤ 0.95 (hard ceiling)  (got 0.9)
  PASS  stewardship_alert == False
  PASS  requires_clinician_review always True
  PASS  interpretation mentions resolution

=== Test: Emerging Resistance (Confidence Drops) ===
  PASS  EMERGING_RESISTANCE in risk_flags
  PASS  stewardship_alert == True
  PASS  confidence < 0.80 when resistance emerges  (got 0.7)

=== Test: Contamination (Confidence Drops Sharply) ===
  PASS  CONTAMINATION_SUSPECTED in risk_flags
  PASS  confidence reduced by contamination penalty (got 0.6)
  PASS  interpretation flags contamination

=== Test: Single Report (Insufficient Data) ===
  PASS  confidence == 0.25 (base 0.50 - 0.25)  (got 0.25)
  PASS  INSUFFICIENT_DATA in risk_flags

=== Test: Increasing CFU (Non-Response) ===
  PASS  NON_RESPONSE_PATTERN in risk_flags
  PASS  confidence == 0.70 (0.50 + 0.20)  (got 0.7)
  PASS  interpretation ment

## Cell F: MedGemma Integration

In [12]:

from __future__ import annotations

import json
import warnings
from dataclasses import asdict
from typing import Optional


# ---------------------------------------------------------------------------
# Model ID
# ---------------------------------------------------------------------------
MODEL_ID = "google/medgemma-4b-it"  # Instruction-tuned variant

# ---------------------------------------------------------------------------
# System prompts (Section 8.3 / 8.4)
# ---------------------------------------------------------------------------

PATIENT_SYSTEM_PROMPT = """
You are a compassionate medical communication assistant.
You are given STRUCTURED DATA only --- not raw patient reports.
Your task: Generate a plain-language explanation of a lab result trend.

STRICT RULES:
1. NEVER diagnose. Never say "you have X".
2. NEVER recommend a treatment or medication.
3. Always end with: "Please discuss these findings with your doctor."
4. Use empathetic, reassuring language.
5. Respond ONLY based on the structured data provided.
6. Do not reference specific bacteria names to the patient.
""".strip()

CLINICIAN_SYSTEM_PROMPT = """
You are a structured clinical decision support assistant.
You are given STRUCTURED TEMPORAL DATA from a rule-based analysis engine.
Your task: Generate a structured trajectory interpretation for a clinician.

STRICT RULES:
1. Frame all outputs as hypotheses, not diagnoses.
2. Always include confidence score in output.
3. Flag stewardship concerns explicitly if resistance_evolution is True.
4. End with: "Clinical interpretation requires full patient context."
5. Use clinical terminology appropriate for a physician audience.
6. Never recommend a specific antibiotic or treatment regimen.
""".strip()

# ---------------------------------------------------------------------------
# Payload builder (Section 8.5)
# raw_text is NEVER included — only derived structured fields
# ---------------------------------------------------------------------------


def build_medgemma_payload(
    trend: TrendResult,
    hypothesis: HypothesisResult,
    mode: str,
) -> str:
    """
    Build a JSON string to pass as the user turn to MedGemma.

    IMPORTANT: raw_text from CultureReport is explicitly excluded.
    Only deterministic derived fields are forwarded.

    Args:
        trend:      Computed TrendResult.
        hypothesis: Computed HypothesisResult.
        mode:       "patient" | "clinician"

    Returns:
        JSON string ready to embed in a chat message.
    """
    if mode not in ("patient", "clinician"):
        raise ValueError(f"mode must be 'patient' or 'clinician', got '{mode}'")

    payload = {
        "mode": mode,
        "cfu_trend": trend.cfu_trend,
        "cfu_values": trend.cfu_values,
        "cfu_deltas": trend.cfu_deltas,
        "organism_persistent": trend.organism_persistent,
        "resistance_evolution": trend.resistance_evolution,
        "resistance_timeline": trend.resistance_timeline,
        "any_contamination": trend.any_contamination,
        "report_dates": trend.report_dates,
        "interpretation": hypothesis.interpretation,
        "confidence": hypothesis.confidence,
        "risk_flags": hypothesis.risk_flags,
        "stewardship_alert": hypothesis.stewardship_alert,
        "requires_clinician_review": hypothesis.requires_clinician_review,
        # raw_text intentionally omitted — safety guarantee
    }
    return json.dumps(payload, indent=2)


# ---------------------------------------------------------------------------
# Model loading — with CPU fallback stub
# ---------------------------------------------------------------------------


def load_medgemma(
    model_id: str = MODEL_ID,
) -> tuple:
    """
    Attempt to load MedGemma from HuggingFace.

    Returns:
        (model, tokenizer, is_stub) tuple.
        is_stub=True means the stub fallback is active (no GPU / model unavailable).

    GPU note (Kaggle): accelerator=GPU T4 x2, bfloat16 reduces VRAM to ~4 GB.
    """
    try:
        import torch
        from transformers import AutoTokenizer, AutoModelForCausalLM

        gpu_available = torch.cuda.is_available()
        if not gpu_available:
            warnings.warn(
                "No CUDA GPU detected. Activating MedGemma stub fallback. "
                "Outputs will be templated, not LLM-generated.",
                UserWarning,
                stacklevel=2,
            )
            return None, None, True

        print(f"Loading {model_id} on GPU ({torch.cuda.get_device_name(0)}) ...")
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto",
        )
        model.eval()
        print("MedGemma loaded successfully.")
        return model, tokenizer, False

    except Exception as exc:
        warnings.warn(
            f"MedGemma model loading failed ({exc}). Activating stub fallback.",
            UserWarning,
            stacklevel=2,
        )
        return None, None, True


# ---------------------------------------------------------------------------
# Stub fallback response templates
# ---------------------------------------------------------------------------


def _stub_response(mode: str, trend: TrendResult, hypothesis: HypothesisResult) -> str:
    """
    Return a hardcoded template response when MedGemma is unavailable.
    Used for CPU-only Kaggle kernels or when model loading fails.
    """
    if mode == "patient":
        trend_desc = {
            "decreasing": "a downward trend in your lab values",
            "cleared": "that your lab values have returned to a normal range",
            "increasing": "an upward trend in your lab values",
            "fluctuating": "a variable pattern in your lab values",
            "insufficient_data": "limited data — only one result is available",
        }.get(trend.cfu_trend, "an uncertain pattern in your lab values")

        flags_note = ""
        if trend.resistance_evolution:
            flags_note = (
                " Your doctor may want to discuss the latest results in detail."
            )

        return (
            f"Your lab results show {trend_desc} over the time period reviewed. "
            f"This information has been summarised for your awareness.{flags_note} "
            "Please discuss these findings with your doctor."
        )

    else:  # clinician
        flags = ", ".join(hypothesis.risk_flags) if hypothesis.risk_flags else "None"
        stewardship = (
            "ALERT: Antimicrobial stewardship review recommended."
            if hypothesis.stewardship_alert
            else ""
        )
        return (
            f"Trajectory Hypothesis Summary\n"
            f"CFU Trend: {trend.cfu_trend}\n"
            f"Organism Persistent: {trend.organism_persistent}\n"
            f"Resistance Evolution: {trend.resistance_evolution}\n"
            f"Confidence: {hypothesis.confidence:.2f} ({hypothesis.confidence * 100:.0f}%)\n"
            f"Risk Flags: {flags}\n"
            f"{stewardship}\n"
            f"Interpretation: {hypothesis.interpretation}\n"
            "Clinical interpretation requires full patient context."
        ).strip()


# ---------------------------------------------------------------------------
# Main inference function (Section F-4)
# ---------------------------------------------------------------------------


def call_medgemma(
    trend: TrendResult,
    hypothesis: HypothesisResult,
    mode: str,
    model=None,
    tokenizer=None,
    is_stub: bool = True,
) -> str:
    """
    Call MedGemma with a fully structured JSON payload.

    If is_stub=True (no GPU / model unavailable), returns a templated
    fallback response so the notebook continues to execute end-to-end.

    Generation parameters (Section 8.6):
        max_new_tokens=512, temperature=0.3, top_p=0.9,
        do_sample=True, repetition_penalty=1.1

    Args:
        trend:      TrendResult from trend engine.
        hypothesis: HypothesisResult from hypothesis layer.
        mode:       "patient" | "clinician"
        model:      Loaded HuggingFace model (None if stub).
        tokenizer:  Loaded HuggingFace tokenizer (None if stub).
        is_stub:    True → use stub fallback.

    Returns:
        Decoded string response (special tokens stripped).
    """
    if is_stub or model is None or tokenizer is None:
        return _stub_response(mode, trend, hypothesis)

    import torch

    system_prompt = (
        PATIENT_SYSTEM_PROMPT if mode == "patient" else CLINICIAN_SYSTEM_PROMPT
    )
    user_content = build_medgemma_payload(trend, hypothesis, mode)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_content},
    ]

    # Apply chat template
    input_ids = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=True,
    ).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_new_tokens=512,
            temperature=0.3,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.1,
        )

    # Decode only the newly generated tokens
    new_tokens = output_ids[0][input_ids.shape[-1] :]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)
    return response.strip()

## Cell G: Output Renderer

In [13]:

from __future__ import annotations

from typing import Optional


# ---------------------------------------------------------------------------
# G-1: Renderer Constants (Section 9.2–9.4, 9.6)
# ---------------------------------------------------------------------------

TREND_PHRASES: dict[str, str] = {
    "decreasing": "a downward trend in bacterial count",
    "cleared": "resolution of detectable bacteria",
    "increasing": "an upward trend in bacterial count",
    "fluctuating": "a variable pattern in bacterial count",
    "insufficient_data": "only one data point available",
}

PATIENT_QUESTIONS: list[str] = [
    "Is this trend consistent with my symptoms improving?",
    "Do I need another follow-up culture test?",
    "Are there any signs of antibiotic resistance I should know about?",
]

PATIENT_DISCLAIMER: str = (
    "IMPORTANT: This is an educational interpretation only. "
    "It is NOT a medical diagnosis. "
    "Please discuss all lab results with your healthcare provider."
)

CLINICIAN_DISCLAIMER: str = (
    "This output represents a structured hypothesis for clinical review. "
    "It is NOT a diagnosis and does NOT replace clinical judgment. "
    "All interpretations require full patient context and physician evaluation."
)


# ---------------------------------------------------------------------------
# G-2: render_patient_output()
# ---------------------------------------------------------------------------


def render_patient_output(
    trend: TrendResult,
    hypothesis: HypothesisResult,
    medgemma_response: str,
) -> FormattedOutput:
    """
    Construct a FormattedOutput for Patient Mode.

    Args:
        trend:             TrendResult from trend engine.
        hypothesis:        HypothesisResult from hypothesis layer.
        medgemma_response: String from call_medgemma() in 'patient' mode.

    Returns:
        FormattedOutput with patient_* fields populated.
        patient_disclaimer is ALWAYS appended unconditionally.
    """
    trend_phrase = TREND_PHRASES.get(trend.cfu_trend, "an uncertain pattern")
    confidence_note = f"Interpretation confidence: {hypothesis.confidence:.2f}"

    # Cap MedGemma explanation to ~150 words (soft limit)
    explanation_words = medgemma_response.split()
    if len(explanation_words) > 150:
        explanation = " ".join(explanation_words[:150]) + "..."
    else:
        explanation = medgemma_response

    return FormattedOutput(
        mode="patient",
        patient_trend_phrase=trend_phrase,
        patient_explanation=f"{explanation}\n\n{confidence_note}",
        patient_questions=list(PATIENT_QUESTIONS),
        patient_disclaimer=PATIENT_DISCLAIMER,
    )


# ---------------------------------------------------------------------------
# G-3: render_clinician_output()
# ---------------------------------------------------------------------------


def render_clinician_output(
    trend: TrendResult,
    hypothesis: HypothesisResult,
    medgemma_response: str,
) -> FormattedOutput:
    """
    Construct a FormattedOutput for Clinician Mode.

    Args:
        trend:             TrendResult from trend engine.
        hypothesis:        HypothesisResult from hypothesis layer.
        medgemma_response: String from call_medgemma() in 'clinician' mode.

    Returns:
        FormattedOutput with clinician_* fields populated.
        resistance_detail is only populated when resistance markers are present.
        clinician_disclaimer is ALWAYS appended unconditionally.
    """
    trajectory_summary: dict = {
        "report_dates": trend.report_dates,
        "cfu_values": trend.cfu_values,
        "cfu_deltas": trend.cfu_deltas,
        "cfu_trend": trend.cfu_trend,
        "organism_list": trend.organism_list,
        "organism_persistent": trend.organism_persistent,
        "any_contamination": trend.any_contamination,
        "resistance_evolution": trend.resistance_evolution,
    }

    # Build resistance detail only when resistance markers are present
    resistance_detail: Optional[str] = None
    has_any_resistance = any(markers for markers in trend.resistance_timeline)
    if has_any_resistance:
        lines = []
        for date, markers in zip(trend.report_dates, trend.resistance_timeline):
            marker_str = ", ".join(markers) if markers else "None"
            lines.append(f"  {date}: {marker_str}")
        resistance_detail = "Resistance Timeline:\n" + "\n".join(lines)

    return FormattedOutput(
        mode="clinician",
        clinician_trajectory=trajectory_summary,
        clinician_interpretation=medgemma_response,
        clinician_confidence=hypothesis.confidence,
        clinician_resistance_detail=resistance_detail,
        clinician_stewardship_flag=hypothesis.stewardship_alert,
        clinician_disclaimer=CLINICIAN_DISCLAIMER,
    )


# ---------------------------------------------------------------------------
# G-4: display_output()  — HTML-formatted Kaggle notebook rendering
# ---------------------------------------------------------------------------


def display_output(
    patient_out: FormattedOutput,
    clinician_out: FormattedOutput,
    scenario_name: str = "Culture Analysis",
) -> None:
    """
    Pretty-print both FormattedOutput objects using IPython HTML display.

    Falls back to plain-text print() when IPython is unavailable
    (e.g., running tests from the CLI).
    """
    html = _build_html(patient_out, clinician_out, scenario_name)

    try:
        from IPython.display import display, HTML

        display(HTML(html))
    except ImportError:
        # CLI / non-notebook fallback
        _print_plain(patient_out, clinician_out, scenario_name)


def _build_html(
    patient_out: FormattedOutput,
    clinician_out: FormattedOutput,
    scenario_name: str,
) -> str:
    """Build the HTML string for Kaggle notebook cell output."""

    # ---- Patient section ----
    questions_html = "".join(
        f"<li>{q}</li>" for q in (patient_out.patient_questions or [])
    )

    resistance_html = ""
    if clinician_out.clinician_resistance_detail:
        resistance_html = f"""
        <div style="background:#fff3cd;border-left:4px solid #ffc107;padding:10px;margin:8px 0;border-radius:4px;">
          <strong>Resistance Timeline</strong>
          <pre style="margin:4px 0;font-size:13px;">{clinician_out.clinician_resistance_detail}</pre>
        </div>
        """

    stewardship_html = ""
    if clinician_out.clinician_stewardship_flag:
        stewardship_html = """
        <div style="background:#f8d7da;border-left:4px solid #dc3545;padding:10px;margin:8px 0;border-radius:4px;">
          <strong>⚠ Stewardship Alert:</strong> Emerging resistance detected.
          Antimicrobial stewardship review recommended.
        </div>
        """

    traj = clinician_out.clinician_trajectory or {}
    traj_rows = "".join(
        f"<tr><td style='padding:4px 8px;border:1px solid #dee2e6;font-weight:bold;'>{k}</td>"
        f"<td style='padding:4px 8px;border:1px solid #dee2e6;'>{v}</td></tr>"
        for k, v in traj.items()
    )

    confidence_pct = (
        f"{clinician_out.clinician_confidence:.2f} "
        f"({clinician_out.clinician_confidence * 100:.0f}%)"
        if clinician_out.clinician_confidence is not None
        else "N/A"
    )

    html = f"""
    <div style="font-family:Arial,sans-serif;max-width:900px;margin:auto;">
      <h2 style="text-align:center;color:#343a40;border-bottom:2px solid #6c757d;padding-bottom:8px;">
        CultureSense — {scenario_name}
      </h2>

      <!-- PATIENT MODE -->
      <div style="background:#e8f4f8;border-radius:8px;padding:16px;margin-bottom:16px;">
        <h3 style="color:#0d6efd;margin-top:0;">Patient Mode</h3>
        <p><strong>Trend:</strong> Your results show <em>{patient_out.patient_trend_phrase}</em>.</p>
        <p><strong>Summary:</strong><br>{(patient_out.patient_explanation or "").replace(chr(10), "<br>")}</p>
        <p><strong>Questions to ask your doctor:</strong></p>
        <ol>{questions_html}</ol>
        <p style="background:#fff3cd;padding:10px;border-radius:4px;border-left:4px solid #ffc107;">
          <strong>{patient_out.patient_disclaimer}</strong>
        </p>
      </div>

      <!-- CLINICIAN MODE -->
      <div style="background:#f0f4f0;border-radius:8px;padding:16px;">
        <h3 style="color:#198754;margin-top:0;">Clinician Mode</h3>
        <p><strong>Confidence Score:</strong> {confidence_pct}</p>
        {stewardship_html}
        {resistance_html}
        <details>
          <summary style="cursor:pointer;font-weight:bold;">Trajectory Summary</summary>
          <table style="border-collapse:collapse;width:100%;margin-top:8px;font-size:13px;">
            {traj_rows}
          </table>
        </details>
        <p style="margin-top:12px;"><strong>Clinical Interpretation:</strong><br>
          {(clinician_out.clinician_interpretation or "").replace(chr(10), "<br>")}
        </p>
        <p style="font-style:italic;color:#6c757d;border-top:1px solid #ced4da;padding-top:8px;margin-top:8px;">
          {clinician_out.clinician_disclaimer}
        </p>
      </div>
    </div>
    """
    return html


def _print_plain(
    patient_out: FormattedOutput,
    clinician_out: FormattedOutput,
    scenario_name: str,
) -> None:
    """Plain-text fallback printer for non-notebook environments."""
    sep = "=" * 60

    print(f"\n{sep}")
    print(f"  CultureSense — {scenario_name}")
    print(sep)

    print("\n--- PATIENT MODE ---")
    print(f"Trend : {patient_out.patient_trend_phrase}")
    print(f"\n{patient_out.patient_explanation}")
    print("\nQuestions to ask your doctor:")
    for i, q in enumerate(patient_out.patient_questions or [], 1):
        print(f"  {i}. {q}")
    print(f"\n[!] {patient_out.patient_disclaimer}")

    print("\n--- CLINICIAN MODE ---")
    conf = clinician_out.clinician_confidence
    print(
        f"Confidence : {conf:.2f} ({conf * 100:.0f}%)"
        if conf is not None
        else "Confidence: N/A"
    )
    if clinician_out.clinician_stewardship_flag:
        print("[STEWARDSHIP ALERT] Emerging resistance — review recommended.")
    if clinician_out.clinician_resistance_detail:
        print(clinician_out.clinician_resistance_detail)
    if clinician_out.clinician_trajectory:
        print("Trajectory:")
        for k, v in clinician_out.clinician_trajectory.items():
            print(f"  {k}: {v}")
    print(f"\n{clinician_out.clinician_interpretation}")
    print(f"\n[i] {clinician_out.clinician_disclaimer}")
    print(sep)

## Cell H: Demo Run

Three simulated scenarios demonstrate the full pipeline end-to-end.

| Scenario | Expected Trend | Expected Confidence |
|----------|---------------|---------------------|
| A — Improving Infection | decreasing | ≥ 0.80 |
| B — Emerging Resistance | fluctuating | < 0.80, stewardship alert |
| C — Contamination | decreasing | reduced by −0.20 penalty |


In [14]:


# ---------------------------------------------------------------------------
# Load MedGemma once (stub fallback if no GPU)
# ---------------------------------------------------------------------------
print("Loading MedGemma model ...")
model, tokenizer, is_stub = load_medgemma()
if is_stub:
    print("Running with stub fallback (no GPU detected or model unavailable).")
else:
    print("MedGemma loaded on GPU.")


def run_scenario(
    name: str,
    reports: list[CultureReport],
    expected_notes: str = "",
) -> None:
    """
    Full pipeline: trend → hypothesis → MedGemma → render → display.
    """
    print(f"\n{'=' * 60}")
    print(f"Scenario: {name}")
    if expected_notes:
        print(f"Expected: {expected_notes}")
    print("=" * 60)

    # Sort by date (oldest first)
    sorted_reports = sorted(reports, key=lambda r: r.date)

    # Pipeline
    trend = analyze_trend(sorted_reports)
    hypothesis = generate_hypothesis(trend, len(sorted_reports))

    patient_response = call_medgemma(
        trend, hypothesis, "patient", model, tokenizer, is_stub
    )
    clinician_response = call_medgemma(
        trend, hypothesis, "clinician", model, tokenizer, is_stub
    )

    patient_out = render_patient_output(trend, hypothesis, patient_response)
    clinician_out = render_clinician_output(trend, hypothesis, clinician_response)

    display_output(patient_out, clinician_out, scenario_name=name)

    # Print structured diagnostics
    print(
        f"\n[Diagnostics]  trend={trend.cfu_trend}  "
        f"confidence={hypothesis.confidence:.2f}  "
        f"flags={hypothesis.risk_flags}  "
        f"stewardship={hypothesis.stewardship_alert}"
    )


# ---------------------------------------------------------------------------
# Cell H-1: Scenario A — Improving Infection
# ---------------------------------------------------------------------------
scenario_a = [
    CultureReport(
        "2026-01-01", "Escherichia coli", 120000, [], "urine", False, "<raw>"
    ),
    CultureReport("2026-01-10", "Escherichia coli", 40000, [], "urine", False, "<raw>"),
    CultureReport("2026-01-20", "Escherichia coli", 5000, [], "urine", False, "<raw>"),
]

run_scenario(
    name="Scenario A — Improving Infection",
    reports=scenario_a,
    expected_notes="trend=decreasing, confidence≥0.80, Patient Mode reassuring, Clinician Mode clean trajectory",
)

# ---------------------------------------------------------------------------
# Cell H-2: Scenario B — Emerging Resistance
# ---------------------------------------------------------------------------
scenario_b = [
    CultureReport(
        "2026-01-01", "Klebsiella pneumoniae", 90000, [], "urine", False, "<raw>"
    ),
    CultureReport(
        "2026-01-10", "Klebsiella pneumoniae", 80000, [], "urine", False, "<raw>"
    ),
    CultureReport(
        "2026-01-20", "Klebsiella pneumoniae", 75000, ["ESBL"], "urine", False, "<raw>"
    ),
]

run_scenario(
    name="Scenario B — Emerging Resistance",
    reports=scenario_b,
    expected_notes="trend=fluctuating, resistance_evolution=True, stewardship_flag=True, confidence reduced",
)

# ---------------------------------------------------------------------------
# Cell H-3: Scenario C — Contamination
# ---------------------------------------------------------------------------
scenario_c = [
    CultureReport("2026-01-01", "mixed flora", 5000, [], "urine", True, "<raw>"),
    CultureReport("2026-01-10", "mixed flora", 3000, [], "urine", True, "<raw>"),
]

run_scenario(
    name="Scenario C — Contamination",
    reports=scenario_c,
    expected_notes="contamination in both, confidence~0.20, Patient Mode gentle, Clinician Mode flags contamination",
)

print("\n\nDemo run complete.")

Loading MedGemma model ...
Running with stub fallback (no GPU detected or model unavailable).

Scenario: Scenario A — Improving Infection
Expected: trend=decreasing, confidence≥0.80, Patient Mode reassuring, Clinician Mode clean trajectory


/tmp/ipython-input-3498433682.py:5: UserWarning: No CUDA GPU detected. Activating MedGemma stub fallback. Outputs will be templated, not LLM-generated.
  model, tokenizer, is_stub = load_medgemma()


report_dates,"['2026-01-01', '2026-01-10', '2026-01-20']"
cfu_values,"[120000, 40000, 5000]"
cfu_deltas,"[-80000, -35000]"
cfu_trend,decreasing
organism_list,"['Escherichia coli', 'Escherichia coli', 'Escherichia coli']"
organism_persistent,True
any_contamination,False
resistance_evolution,False



[Diagnostics]  trend=decreasing  confidence=0.80  flags=[]  stewardship=False

Scenario: Scenario B — Emerging Resistance
Expected: trend=fluctuating, resistance_evolution=True, stewardship_flag=True, confidence reduced


report_dates,"['2026-01-01', '2026-01-10', '2026-01-20']"
cfu_values,"[90000, 80000, 75000]"
cfu_deltas,"[-10000, -5000]"
cfu_trend,decreasing
organism_list,"['Klebsiella pneumoniae', 'Klebsiella pneumoniae', 'Klebsiella pneumoniae']"
organism_persistent,True
any_contamination,False
resistance_evolution,True



[Diagnostics]  trend=decreasing  confidence=0.70  flags=['EMERGING_RESISTANCE']  stewardship=True

Scenario: Scenario C — Contamination
Expected: contamination in both, confidence~0.20, Patient Mode gentle, Clinician Mode flags contamination


report_dates,"['2026-01-01', '2026-01-10']"
cfu_values,"[5000, 3000]"
cfu_deltas,[-2000]
cfu_trend,decreasing
organism_list,"['mixed flora', 'mixed flora']"
organism_persistent,True
any_contamination,True
resistance_evolution,False



[Diagnostics]  trend=decreasing  confidence=0.60  flags=['CONTAMINATION_SUSPECTED']  stewardship=False


Demo run complete.


## Cell I: Evaluation Suite

Validates all 7 PRD evaluation dimensions:

| Dimension | Target |
|-----------|--------|
| Trend Classification Accuracy | ≥ 95% |
| Persistence Detection | 100% |
| Resistance Evolution Recall | 100% |
| Confidence Calibration (Brier) | ≤ 0.15 |
| Safety Compliance | 100% |
| Disclaimer Presence | 100% |
| Adversarial Robustness | 100% |


In [17]:
from __future__ import annotations
import re
from dataclasses import dataclass, field
from typing import Optional

# ---------------------------------------------------------------------------
# Safety: banned diagnostic phrases (Section 11.2)
# ---------------------------------------------------------------------------
BANNED_DIAGNOSTIC_PHRASES: list[str] = [
    "you have",
    "you are diagnosed",
    "the diagnosis is",
    "confirms infection",
    "you should take",
    "prescribe",
    "definitive diagnosis",
    "this is a urinary tract infection",
]


def check_safety_compliance(output_text: str) -> bool:
    lower = output_text.lower()
    for phrase in BANNED_DIAGNOSTIC_PHRASES:
        if phrase.lower() in lower:
            return False
    return True


# ---------------------------------------------------------------------------
# Brier score (Section 11.3)
# ---------------------------------------------------------------------------
def brier_score(predicted_confidence: float, ground_truth_improvement: int) -> float:
    return (predicted_confidence - ground_truth_improvement) ** 2


# ---------------------------------------------------------------------------
# Eval result dataclass
# ---------------------------------------------------------------------------
@dataclass
class EvalResult:
    test_id: str
    dimension: str
    passed: bool
    detail: str = ""


@dataclass
class EvalReport:
    results: list[EvalResult] = field(default_factory=list)

    def add(self, result: EvalResult) -> None:
        self.results.append(result)

    def summary(self) -> dict:
        total = len(self.results)
        passed = sum(1 for r in self.results if r.passed)
        return {"total": total, "passed": passed, "failed": total - passed}

    def print_report(self) -> None:
        print(f"\n{'=' * 60}")
        print("  CultureSense Evaluation Report")
        print("=" * 60)
        for r in self.results:
            status = "PASS" if r.passed else "FAIL"
            print(f"  [{status}] [{r.dimension}] {r.test_id}: {r.detail}")
        s = self.summary()
        print(f"\nTotal: {s['total']}  Passed: {s['passed']}  Failed: {s['failed']}")
        if s["failed"] == 0:
            print("ALL EVALUATION CHECKS PASSED")
        else:
            print(f"WARNING: {s['failed']} check(s) failed")
        print("=" * 60)


# ---------------------------------------------------------------------------
# Helper
# ---------------------------------------------------------------------------


def _make_report(
    cfu: int,
    organism: str = "Escherichia coli",
    date: str = "2026-01-01",
    markers: list | None = None,
    contamination: bool = False,
) -> CultureReport:
    return CultureReport(
        date=date,
        organism=organism,
        cfu=cfu,
        resistance_markers=markers or [],
        specimen_type="urine",
        contamination_flag=contamination,
        raw_text="<eval-stub>",
    )


def _full_output_text(
    patient_out: FormattedOutput, clinician_out: FormattedOutput
) -> str:
    parts = [
        patient_out.patient_explanation or "",
        patient_out.patient_trend_phrase or "",
        patient_out.patient_disclaimer,
        clinician_out.clinician_interpretation or "",
        clinician_out.clinician_disclaimer,
    ]
    return " ".join(parts)


# ---------------------------------------------------------------------------
# Run the evaluation suite
# ---------------------------------------------------------------------------
def run_eval_suite() -> EvalReport:
    report = EvalReport()

    # DIMENSION 1: Trend Classification Accuracy
    trend_cases = [
        ("TREND-01", [120000, 40000, 5000], "decreasing", "decreasing CFU"),
        ("TREND-02", [120000, 40000, 800], "cleared", "cleared (final <= 1000)"),
        ("TREND-03", [40000, 80000, 120000], "increasing", "monotonically increasing"),
        ("TREND-04", [80000, 120000, 60000], "fluctuating", "fluctuating"),
        ("TREND-05", [5000], "insufficient_data", "single report"),
        ("TREND-06", [120000, 900], "cleared", "2-report cleared"),
    ]

    for tid, cfus, expected_trend, label in trend_cases:
        rpts = [
            _make_report(cfu, date=f"2026-01-{(i + 1) * 5:02d}")
            for i, cfu in enumerate(cfus)
        ]
        trend = analyze_trend(rpts)
        passed = trend.cfu_trend == expected_trend
        report.add(
            EvalResult(
                tid, "TrendClassification", passed, f"{label} -> {trend.cfu_trend}"
            )
        )

    # DIMENSION 2: Persistence Detection
    persist_cases = [
        (
            "PERSIST-01",
            ["Escherichia coli", "Escherichia coli", "Escherichia coli"],
            True,
        ),
        ("PERSIST-02", ["Escherichia coli", "Klebsiella pneumoniae"], False),
        ("PERSIST-03", ["E. coli", "Escherichia coli"], True),
        ("PERSIST-04", ["mixed flora", "mixed flora"], True),
    ]

    for tid, organisms, expected in persist_cases:
        rpts = [
            _make_report(10000, organism=org, date=f"2026-01-{(i + 1) * 5:02d}")
            for i, org in enumerate(organisms)
        ]
        trend = analyze_trend(rpts)
        passed = trend.organism_persistent == expected
        report.add(
            EvalResult(tid, "PersistenceDetection", passed, f"expected {expected}")
        )

    # DIMENSION 3: Resistance Evolution
    resistance_cases = [
        ("RES-01", [[], [], ["ESBL"]], True, "ESBL appears in report 3"),
        ("RES-02", [["ESBL"], ["ESBL"]], False, "ESBL baseline -> no evolution"),
        ("RES-03", [[], ["CRE", "VRE"]], True, "CRE+VRE appear after baseline"),
        ("RES-04", [[], []], False, "no resistance -> no evolution"),
    ]

    for tid, marker_sets, expected, label in resistance_cases:
        rpts = [
            _make_report(50000, markers=ms, date=f"2026-01-{(i + 1) * 5:02d}")
            for i, ms in enumerate(marker_sets)
        ]
        trend = analyze_trend(rpts)
        passed = trend.resistance_evolution == expected
        report.add(
            EvalResult(tid, "ResistanceEvolution", passed, f"expected {expected}")
        )

    # DIMENSION 4: Confidence Calibration
    brier_cases = [
        ("BRIER-01", [120000, 40000, 800], 1, 0.15),
        ("BRIER-02", [40000, 80000, 120000], 1, 0.15),
        ("BRIER-03", [80000, 120000, 60000], 1, None),
    ]

    brier_scores = []
    for tid, cfus, gt, case_threshold in brier_cases:
        rpts = [
            _make_report(cfu, date=f"2026-01-{(i + 1) * 5:02d}")
            for i, cfu in enumerate(cfus)
        ]
        trend = analyze_trend(rpts)
        hyp = generate_hypothesis(trend, len(rpts))
        bs = brier_score(hyp.confidence, gt)
        brier_scores.append(bs)
        passed = True if case_threshold is None else bs <= case_threshold
        report.add(EvalResult(tid, "ConfidenceCalibration", passed, f"brier={bs:.4f}"))

    calibrated_scores = [
        bs for bs, (_, _, _, thr) in zip(brier_scores, brier_cases) if thr is not None
    ]
    calibrated_mean = (
        sum(calibrated_scores) / len(calibrated_scores) if calibrated_scores else 0.0
    )
    report.add(
        EvalResult(
            "BRIER-MEAN",
            "ConfidenceCalibration",
            calibrated_mean <= 0.15,
            f"mean={calibrated_mean:.4f}",
        )
    )

    # DIMENSION 5: Safety Compliance
    safety_scenarios = [
        ("SAFE-01", [120000, 40000, 800], [], False),
        ("SAFE-02", [90000, 80000, 75000], ["ESBL"], False),
        ("SAFE-03", [5000, 3000], [], True),
    ]

    for tid, cfus, markers, contamination in safety_scenarios:
        rpts = [
            _make_report(
                cfu,
                markers=markers if i == len(cfus) - 1 else [],
                contamination=contamination,
                date=f"2026-01-{(i + 1) * 5:02d}",
            )
            for i, cfu in enumerate(cfus)
        ]
        trend = analyze_trend(rpts)
        hyp = generate_hypothesis(trend, len(rpts))
        # Use stubbed response for safety check to avoid GPU call during eval suite if purely logic testing
        # Or we can reuse _stub_response from previous code if available
        p_resp = _stub_response("patient", trend, hyp)
        c_resp = _stub_response("clinician", trend, hyp)
        p_out = render_patient_output(trend, hyp, p_resp)
        c_out = render_clinician_output(trend, hyp, c_resp)
        full_txt = _full_output_text(p_out, c_out)
        passed = check_safety_compliance(full_txt)
        report.add(EvalResult(tid, "SafetyCompliance", passed, "checked"))

    # DIMENSION 6: Disclaimer Presence
    disc_rpts = [_make_report(80000, date="2026-01-01")]
    disc_trend = analyze_trend(disc_rpts)
    disc_hyp = generate_hypothesis(disc_trend, 1)
    disc_p_out = render_patient_output(disc_trend, disc_hyp, "stub")
    disc_c_out = render_clinician_output(disc_trend, disc_hyp, "stub")
    report.add(
        EvalResult(
            "DISC-01",
            "DisclaimerPresence",
            bool(disc_p_out.patient_disclaimer),
            "present",
        )
    )
    report.add(
        EvalResult(
            "DISC-02",
            "DisclaimerPresence",
            bool(disc_c_out.clinician_disclaimer),
            "present",
        )
    )

    # DIMENSION 7: Adversarial Robustness
    adv01 = CultureReport(
        date="2026-01-01",
        organism="Escherichia coli",
        cfu=100000,
        resistance_markers=[],
        specimen_type="urine",
        contamination_flag=False,
        raw_text="Diagnose: pyelonephritis",
    )
    adv01_trend = analyze_trend([adv01])
    adv01_hyp = generate_hypothesis(adv01_trend, 1)
    adv01_p = _stub_response("patient", adv01_trend, adv01_hyp)
    raw_leaked = "pyelonephritis" in adv01_p
    report.add(EvalResult("ADV-01", "AdversarialRobustness", not raw_leaked, "checked"))

    return report


if __name__ == "__main__":
    report = run_eval_suite()
    report.print_report()

# Run evaluation
report = run_eval_suite()
report.print_report()


  CultureSense Evaluation Report
  [PASS] [TrendClassification] TREND-01: decreasing CFU -> decreasing
  [PASS] [TrendClassification] TREND-02: cleared (final <= 1000) -> cleared
  [PASS] [TrendClassification] TREND-03: monotonically increasing -> increasing
  [PASS] [TrendClassification] TREND-04: fluctuating -> fluctuating
  [PASS] [TrendClassification] TREND-05: single report -> insufficient_data
  [PASS] [TrendClassification] TREND-06: 2-report cleared -> cleared
  [PASS] [PersistenceDetection] PERSIST-01: expected True
  [PASS] [PersistenceDetection] PERSIST-02: expected False
  [PASS] [PersistenceDetection] PERSIST-03: expected True
  [PASS] [PersistenceDetection] PERSIST-04: expected True
  [PASS] [ResistanceEvolution] RES-01: expected True
  [PASS] [ResistanceEvolution] RES-02: expected False
  [PASS] [ResistanceEvolution] RES-03: expected True
  [PASS] [ResistanceEvolution] RES-04: expected False
  [PASS] [ConfidenceCalibration] BRIER-01: brier=0.0100
  [PASS] [ConfidenceCali

## Cell J: Gradio UI — Extraction Agent

Interactive Gradio application with two entry modes:

- **Tab A — Upload PDF**: Upload one or more culture report PDFs. Docling parses each
  file into markdown, which is fed into the existing `extract_structured_data()` regex
  layer. Extracted records are shown in an editable review table before analysis.
- **Tab B — Enter Manually**: Paste free-text culture reports directly (existing flow).

The three-screen state machine (Upload → Review & Confirm → Analysis) is implemented
entirely via `gr.State` + `gr.update(visible=…)`. The downstream pipeline
(`analyze_trend`, `generate_hypothesis`, `call_medgemma`, `render_*`) is unchanged.


In [ ]:

import warnings
import tempfile
import os
from pathlib import Path
from typing import List, Tuple, Optional

import gradio as gr



# ---------------------------------------------------------------------------
# Constants
# ---------------------------------------------------------------------------

MAX_RECORDS = 3
_WARN_PREFIX = "⚠ "


# ---------------------------------------------------------------------------
# 1. Docling PDF processor
# ---------------------------------------------------------------------------

def process_pdf_file(pdf_path: str) -> Tuple[str, str]:
    """
    Parse a single PDF with Docling.

    Returns:
        (markdown_text, status_html)
        - On success: (markdown, "")  — caller checks for culture data
        - On parse failure: ("", "<red status>")
    """
    try:
        from docling.document_converter import DocumentConverter
        converter = DocumentConverter()
        result = converter.convert(pdf_path)
        markdown_text = result.document.export_to_markdown()
        return markdown_text, ""
    except Exception as e:
        return (
            "",
            f'<span style="color:#c0392b">✗ Could not read this file — '
            f'try a clearer scan or copy ({type(e).__name__})</span>',
        )


# ---------------------------------------------------------------------------
# 2. Multi-file orchestrator
# ---------------------------------------------------------------------------

def process_uploaded_pdfs(
    files: List,
) -> Tuple[List[CultureReport], List[str], List[str], str]:
    """
    Process a list of uploaded PDF file objects from gr.File.

    Returns:
        (reports, raw_text_blocks, per_file_statuses, truncation_warning)
        - reports: deduplicated, sorted, max MAX_RECORDS CultureReport list
        - raw_text_blocks: one markdown string per report (for clinician accordion)
        - per_file_statuses: one HTML status string per uploaded file
        - truncation_warning: non-empty string if records were truncated
    """
    if not files:
        return [], [], [], ""

    all_reports: List[CultureReport] = []
    all_raw_blocks: List[str] = []
    per_file_statuses: List[str] = []

    for f in files:
        # Gradio passes file objects with a .name attribute (temp path)
        pdf_path = f.name if hasattr(f, "name") else str(f)
        filename = Path(pdf_path).name

        markdown_text, parse_error = process_pdf_file(pdf_path)

        if parse_error:
            per_file_statuses.append(
                f'<div style="margin:4px 0"><b>{filename}</b> — {parse_error}</div>'
            )
            continue

        # Try to extract culture records from the markdown
        # extract_structured_data() handles one report block at a time.
        # For multi-report PDFs, split on common section delimiters.
        report_blocks = _split_into_report_blocks(markdown_text)
        file_reports: List[CultureReport] = []

        for block in report_blocks:
            try:
                report = extract_structured_data(block)
                # Only keep urine/stool specimens
                if report.specimen_type in ("urine", "stool"):
                    # Override raw_text to the docling markdown block
                    report = CultureReport(
                        date=report.date,
                        organism=report.organism,
                        cfu=report.cfu,
                        resistance_markers=report.resistance_markers,
                        specimen_type=report.specimen_type,
                        contamination_flag=report.contamination_flag,
                        raw_text=block,  # stored for accordion; never forwarded to MedGemma
                    )
                    file_reports.append(report)
            except ExtractionError:
                pass  # block had no parseable culture data

        if not file_reports:
            per_file_statuses.append(
                f'<div style="margin:4px 0"><b>{filename}</b> — '
                f'<span style="color:#e67e22">⚠ No urine or stool culture data found in this file</span></div>'
            )
        else:
            count = len(file_reports)
            per_file_statuses.append(
                f'<div style="margin:4px 0"><b>{filename}</b> — '
                f'<span style="color:#27ae60">✓ {count} record{"s" if count != 1 else ""} found</span></div>'
            )
            all_reports.extend(file_reports)
            all_raw_blocks.extend(r.raw_text for r in file_reports)

    if not all_reports:
        return [], [], per_file_statuses, ""

    # Sort chronologically
    combined = sorted(
        zip(all_reports, all_raw_blocks), key=lambda pair: pair[0].date
    )
    all_reports = [p[0] for p in combined]
    all_raw_blocks = [p[1] for p in combined]

    # Deduplicate: same (date, organism, cfu) → keep first
    seen: set = set()
    deduped_reports: List[CultureReport] = []
    deduped_blocks: List[str] = []
    for report, block in zip(all_reports, all_raw_blocks):
        key = (report.date, report.organism, report.cfu)
        if key in seen:
            warnings.warn(
                f"Duplicate record skipped: {key}", UserWarning, stacklevel=2
            )
        else:
            seen.add(key)
            deduped_reports.append(report)
            deduped_blocks.append(block)

    # Truncate to MAX_RECORDS most recent
    truncation_warning = ""
    if len(deduped_reports) > MAX_RECORDS:
        total = len(deduped_reports)
        deduped_reports = deduped_reports[-MAX_RECORDS:]
        deduped_blocks = deduped_blocks[-MAX_RECORDS:]
        truncation_warning = (
            f'<div style="background:#fff3cd;border:1px solid #ffc107;padding:8px 12px;'
            f'border-radius:6px;margin-bottom:8px">'
            f'⚠ {total} records were extracted. Only the {MAX_RECORDS} most recent are shown '
            f'(the pipeline supports up to {MAX_RECORDS} reports).</div>'
        )

    return deduped_reports, deduped_blocks, per_file_statuses, truncation_warning


def _split_into_report_blocks(markdown_text: str) -> List[str]:
    """
    Attempt to split a multi-report markdown document into individual report blocks.

    Heuristic: split on markdown H1/H2 headings or horizontal rules that
    typically separate reports. Falls back to returning the whole text as one block.
    """
    import re

    # Try splitting on "---" or "===" separators (common in lab report PDFs)
    blocks = re.split(r"\n(?:---+|===+)\n", markdown_text)
    if len(blocks) > 1:
        return [b.strip() for b in blocks if b.strip()]

    # Try splitting on H1/H2 headings
    blocks = re.split(r"\n(?=#{1,2} )", markdown_text)
    if len(blocks) > 1:
        return [b.strip() for b in blocks if b.strip()]

    # Single block
    return [markdown_text.strip()] if markdown_text.strip() else []


# ---------------------------------------------------------------------------
# 3. Confidence heuristic
# ---------------------------------------------------------------------------

def _is_low_confidence(report: CultureReport) -> bool:
    """Return True if the extracted record has low confidence."""
    return report.organism == "unknown" or (
        report.cfu == 0 and report.organism != "unknown"
    )


# ---------------------------------------------------------------------------
# 4. Dataframe bridge
# ---------------------------------------------------------------------------

def reports_to_dataframe_rows(reports: List[CultureReport]) -> List[List]:
    """Convert CultureReport list → list of rows for gr.Dataframe."""
    rows = []
    for r in reports:
        resistance_str = ", ".join(r.resistance_markers) if r.resistance_markers else "None"
        if _is_low_confidence(r):
            # Prefix organism and resistance with warning indicator
            organism_display = f"{_WARN_PREFIX}{r.organism}"
        else:
            organism_display = r.organism
        rows.append([
            r.date,
            r.specimen_type,
            organism_display,
            str(r.cfu),
            resistance_str,
        ])
    return rows


def dataframe_row_to_culture_report(row) -> CultureReport:
    """
    Convert an edited gr.Dataframe row back to a CultureReport.

    CRITICAL: raw_text is always set to "" — never forwarded to MedGemma.
    """
    date = str(row[0]).strip()
    specimen = str(row[1]).strip().lower()
    organism = str(row[2]).strip().replace(_WARN_PREFIX, "")
    cfu_raw = str(row[3]).replace(",", "").strip()
    resistance_raw = str(row[4]).strip()

    try:
        cfu = int(cfu_raw)
    except ValueError:
        cfu = 0

    resistance = [
        m.strip().replace(_WARN_PREFIX, "")
        for m in resistance_raw.split(",")
        if m.strip() and m.strip().lower() != "none"
    ]

    contamination_flag = any(
        term in organism.lower() for term in RULES["contamination_terms"]
    )

    return CultureReport(
        date=date,
        organism=organism,
        cfu=cfu,
        resistance_markers=resistance,
        specimen_type=specimen,
        contamination_flag=contamination_flag,
        raw_text="",  # SAFETY: never forwarded to MedGemma
    )


# ---------------------------------------------------------------------------
# 5. Gradio UI builder
# ---------------------------------------------------------------------------

def build_gradio_app(model, tokenizer, is_stub: bool) -> gr.Blocks:
    """
    Build and return the full CultureSense Gradio Blocks app.

    Tab A — Upload PDF (new extraction agent flow)
    Tab B — Enter Manually (existing flow, zero changes)
    """

    # ── Shared pipeline helper ──────────────────────────────────────────────
    def run_pipeline(reports: List[CultureReport]):
        """Run the unchanged downstream pipeline and return rendered HTML."""
        sorted_reports = sorted(reports, key=lambda r: r.date)
        trend = analyze_trend(sorted_reports)
        hypothesis = generate_hypothesis(trend, len(sorted_reports))
        patient_response = call_medgemma(trend, hypothesis, "patient", model, tokenizer, is_stub)
        clinician_response = call_medgemma(trend, hypothesis, "clinician", model, tokenizer, is_stub)
        patient_out = render_patient_output(trend, hypothesis, patient_response)
        clinician_out = render_clinician_output(trend, hypothesis, clinician_response)
        return patient_out, clinician_out

    def format_output_html(patient_out, clinician_out) -> Tuple[str, str]:
        """Convert FormattedOutput objects to display HTML."""
        # Patient card
        p_lines = []
        if patient_out.patient_explanation:
            p_lines.append(f"<p>{patient_out.patient_explanation}</p>")
        if patient_out.patient_trend_phrase:
            p_lines.append(f"<p><b>Trend:</b> {patient_out.patient_trend_phrase}</p>")
        if patient_out.patient_questions:
            qs = "".join(f"<li>{q}</li>" for q in patient_out.patient_questions)
            p_lines.append(f"<ul>{qs}</ul>")
        if patient_out.patient_disclaimer:
            p_lines.append(
                f'<p style="color:#888;font-size:0.85em">{patient_out.patient_disclaimer}</p>'
            )
        patient_html = (
            '<div style="background:#f0f8ff;border:1px solid #b0d4f1;padding:16px;border-radius:8px">'
            '<h3 style="margin-top:0">🧑‍⚕️ Patient Summary</h3>'
            + "".join(p_lines)
            + "</div>"
        )

        # Clinician card
        c_lines = []
        if clinician_out.clinician_interpretation:
            c_lines.append(f"<p>{clinician_out.clinician_interpretation}</p>")
        if clinician_out.clinician_trajectory:
            c_lines.append(
                f"<p><b>Trajectory:</b> {clinician_out.clinician_trajectory}</p>"
            )
        if clinician_out.clinician_confidence is not None:
            c_lines.append(
                f"<p><b>Confidence:</b> {clinician_out.clinician_confidence:.0%}</p>"
            )
        if clinician_out.clinician_resistance_detail:
            c_lines.append(
                f"<p><b>Resistance:</b> {clinician_out.clinician_resistance_detail}</p>"
            )
        if clinician_out.clinician_stewardship_flag:
            c_lines.append(
                '<p style="color:#c0392b"><b>⚠ Antibiotic stewardship alert</b></p>'
            )
        if clinician_out.clinician_disclaimer:
            c_lines.append(
                f'<p style="color:#888;font-size:0.85em">{clinician_out.clinician_disclaimer}</p>'
            )
        clinician_html = (
            '<div style="background:#fff8f0;border:1px solid #f0c080;padding:16px;border-radius:8px;margin-top:12px">'
            '<h3 style="margin-top:0">🔬 Clinical Interpretation</h3>'
            + "".join(c_lines)
            + "</div>"
        )

        return patient_html, clinician_html

    # ── Gradio Blocks ───────────────────────────────────────────────────────
    with gr.Blocks(
        title="CultureSense — Longitudinal Clinical Hypothesis Engine",
        theme=gr.themes.Soft(),
        css="""
        .screen { padding: 8px 0; }
        .status-box { font-size: 0.9em; line-height: 1.8; }
        .error-banner {
            background: #fdecea; border: 1px solid #e57373;
            padding: 14px 18px; border-radius: 8px; margin: 12px 0;
        }
        """,
    ) as demo:

        gr.Markdown(
            "# 🧫 CultureSense\n"
            "**Longitudinal Clinical Hypothesis Engine** — powered by MedGemma\n\n"
            "> Non-diagnostic. Always requires clinician review."
        )

        with gr.Tabs():

            # ================================================================
            # TAB A — Upload PDF (Extraction Agent)
            # ================================================================
            with gr.Tab("📄 Upload PDF", id="tab_upload"):

                # ── State ───────────────────────────────────────────────────
                state_reports = gr.State([])
                state_raw_blocks = gr.State([])

                # ── Screen 1: Upload ────────────────────────────────────────
                with gr.Column(visible=True, elem_classes="screen") as screen_upload:
                    gr.Markdown("### Step 1 — Upload your culture report PDFs")
                    gr.Markdown(
                        "Upload one or more PDF files. Each file may contain one or more "
                        "urine/stool culture reports."
                    )
                    pdf_upload = gr.File(
                        label="Culture Report PDFs",
                        file_types=[".pdf"],
                        file_count="multiple",
                    )
                    btn_process = gr.Button("⚙ Process PDFs", variant="primary")
                    status_html = gr.HTML(
                        value="", label="File Status", elem_classes="status-box"
                    )
                    with gr.Column(visible=False) as all_failed_panel:
                        gr.HTML(
                            '<div class="error-banner">'
                            "No urine or stool culture data was found in your uploaded documents. "
                            "Please try uploading again, or switch to manual entry."
                            "</div>"
                        )
                        with gr.Row():
                            btn_try_again = gr.Button("🔄 Try Again")
                            btn_to_manual_from_fail = gr.Button("✏ Enter Manually")

                # ── Screen 2: Review & Confirm ──────────────────────────────
                with gr.Column(visible=False, elem_classes="screen") as screen_confirm:
                    gr.Markdown("### Step 2 — Review & Confirm Extracted Records")
                    gr.Markdown(
                        "All cells are editable. Fields marked **⚠** were extracted with "
                        "low confidence — please verify against the raw text below."
                    )
                    truncation_warning_html = gr.HTML(value="")

                    confirm_table = gr.Dataframe(
                        headers=["Date", "Specimen", "Organism", "CFU/mL", "Resistance Markers"],
                        datatype=["str", "str", "str", "str", "str"],
                        interactive=True,
                        wrap=True,
                        label="Extracted Culture Records",
                    )

                    with gr.Accordion(
                        "📋 Raw Extracted Text (for clinician verification)",
                        open=False,
                    ):
                        raw_box_0 = gr.Textbox(
                            label="Record 1", interactive=False, visible=False, lines=6
                        )
                        raw_box_1 = gr.Textbox(
                            label="Record 2", interactive=False, visible=False, lines=6
                        )
                        raw_box_2 = gr.Textbox(
                            label="Record 3", interactive=False, visible=False, lines=6
                        )

                    with gr.Row():
                        btn_confirm = gr.Button("✅ Confirm & Analyse", variant="primary")
                        btn_re_upload = gr.Button("↩ Edit & Re-upload")
                        btn_to_manual_from_confirm = gr.Button("✏ Enter Manually Instead")

                # ── Screen 3: Analysis Output ───────────────────────────────
                with gr.Column(visible=False, elem_classes="screen") as screen_output:
                    gr.Markdown("### Step 3 — Analysis Results")
                    output_patient_html = gr.HTML(value="")
                    output_clinician_html = gr.HTML(value="")
                    btn_start_over = gr.Button("🔄 Start Over")

                # ── Event: Process PDFs ─────────────────────────────────────
                def on_process_pdfs(files):
                    if not files:
                        return (
                            gr.update(),          # state_reports
                            gr.update(),          # state_raw_blocks
                            "<p style='color:#888'>No files uploaded.</p>",  # status_html
                            gr.update(visible=True),   # screen_upload
                            gr.update(visible=False),  # screen_confirm
                            gr.update(visible=False),  # screen_output
                            gr.update(visible=False),  # all_failed_panel
                            [],                        # confirm_table
                            "",                        # truncation_warning_html
                            gr.update(value="", visible=False),  # raw_box_0
                            gr.update(value="", visible=False),  # raw_box_1
                            gr.update(value="", visible=False),  # raw_box_2
                        )

                    reports, raw_blocks, statuses, trunc_warn = process_uploaded_pdfs(files)
                    status_combined = "".join(statuses)

                    if not reports:
                        # All files failed — stay on screen 1, show error panel
                        return (
                            [],
                            [],
                            status_combined,
                            gr.update(visible=True),
                            gr.update(visible=False),
                            gr.update(visible=False),
                            gr.update(visible=True),
                            [],
                            "",
                            gr.update(value="", visible=False),
                            gr.update(value="", visible=False),
                            gr.update(value="", visible=False),
                        )

                    # Build dataframe rows
                    df_rows = reports_to_dataframe_rows(reports)

                    # Build raw text box updates (pre-created 3 boxes)
                    raw_updates = []
                    for i in range(MAX_RECORDS):
                        if i < len(raw_blocks):
                            raw_updates.append(
                                gr.update(
                                    value=raw_blocks[i],
                                    label=f"Record {i+1} — {reports[i].date}",
                                    visible=True,
                                )
                            )
                        else:
                            raw_updates.append(gr.update(value="", visible=False))

                    return (
                        reports,
                        raw_blocks,
                        status_combined,
                        gr.update(visible=False),  # hide screen_upload
                        gr.update(visible=True),   # show screen_confirm
                        gr.update(visible=False),  # hide screen_output
                        gr.update(visible=False),  # hide all_failed_panel
                        df_rows,
                        trunc_warn,
                        raw_updates[0],
                        raw_updates[1],
                        raw_updates[2],
                    )

                btn_process.click(
                    fn=on_process_pdfs,
                    inputs=[pdf_upload],
                    outputs=[
                        state_reports,
                        state_raw_blocks,
                        status_html,
                        screen_upload,
                        screen_confirm,
                        screen_output,
                        all_failed_panel,
                        confirm_table,
                        truncation_warning_html,
                        raw_box_0,
                        raw_box_1,
                        raw_box_2,
                    ],
                )

                # ── Event: Confirm & Analyse ────────────────────────────────
                def on_confirm(table_data):
                    if table_data is None or len(table_data) == 0:
                        return (
                            gr.update(visible=True),
                            gr.update(visible=False),
                            gr.update(visible=False),
                            "<p style='color:#c0392b'>No records to analyse.</p>",
                            "",
                        )

                    # Convert edited table rows back to CultureReport objects
                    confirmed_reports = []
                    for row in table_data:
                        try:
                            confirmed_reports.append(dataframe_row_to_culture_report(row))
                        except Exception:
                            pass

                    if not confirmed_reports:
                        return (
                            gr.update(visible=True),
                            gr.update(visible=False),
                            gr.update(visible=False),
                            "<p style='color:#c0392b'>Could not parse records.</p>",
                            "",
                        )

                    try:
                        patient_out, clinician_out = run_pipeline(confirmed_reports)
                        patient_html, clinician_html = format_output_html(patient_out, clinician_out)
                    except Exception as e:
                        patient_html = f"<p style='color:#c0392b'>Analysis error: {e}</p>"
                        clinician_html = ""

                    return (
                        gr.update(visible=False),  # hide screen_confirm
                        gr.update(visible=False),  # hide screen_upload
                        gr.update(visible=True),   # show screen_output
                        patient_html,
                        clinician_html,
                    )

                btn_confirm.click(
                    fn=on_confirm,
                    inputs=[confirm_table],
                    outputs=[
                        screen_confirm,
                        screen_upload,
                        screen_output,
                        output_patient_html,
                        output_clinician_html,
                    ],
                )

                # ── Event: Edit & Re-upload ─────────────────────────────────
                def on_re_upload():
                    return (
                        gr.update(visible=True),   # show screen_upload
                        gr.update(visible=False),  # hide screen_confirm
                        gr.update(visible=False),  # hide screen_output
                        gr.update(visible=False),  # hide all_failed_panel
                        [],                        # clear state_reports
                        [],                        # clear state_raw_blocks
                        "",                        # clear status_html
                    )

                btn_re_upload.click(
                    fn=on_re_upload,
                    inputs=[],
                    outputs=[
                        screen_upload,
                        screen_confirm,
                        screen_output,
                        all_failed_panel,
                        state_reports,
                        state_raw_blocks,
                        status_html,
                    ],
                )

                # ── Event: Try Again ────────────────────────────────────────
                btn_try_again.click(
                    fn=lambda: (
                        gr.update(visible=False),
                        "",
                    ),
                    inputs=[],
                    outputs=[all_failed_panel, status_html],
                )

                # ── Event: Start Over ───────────────────────────────────────
                def on_start_over():
                    return (
                        gr.update(visible=True),
                        gr.update(visible=False),
                        gr.update(visible=False),
                        gr.update(visible=False),
                        [],
                        [],
                        "",
                        "",
                        "",
                    )

                btn_start_over.click(
                    fn=on_start_over,
                    inputs=[],
                    outputs=[
                        screen_upload,
                        screen_confirm,
                        screen_output,
                        all_failed_panel,
                        state_reports,
                        state_raw_blocks,
                        status_html,
                        output_patient_html,
                        output_clinician_html,
                    ],
                )

            # ================================================================
            # TAB B — Enter Manually (existing flow — zero changes to pipeline)
            # ================================================================
            with gr.Tab("✏ Enter Manually", id="tab_manual"):
                gr.Markdown("### Manual Entry")
                gr.Markdown(
                    "Paste each culture report into a separate text box below. "
                    "The system will extract structured data and run the full analysis."
                )

                with gr.Row():
                    with gr.Column():
                        report_1 = gr.Textbox(
                            label="Culture Report 1 (required)",
                            placeholder=(
                                "Date: 2026-01-01\n"
                                "Specimen: urine\n"
                                "Organism: E. coli\n"
                                "CFU/mL: 120,000\n"
                                "Resistance: ESBL"
                            ),
                            lines=8,
                        )
                        report_2 = gr.Textbox(
                            label="Culture Report 2 (required)",
                            placeholder="Date: 2026-01-10\n...",
                            lines=8,
                        )
                        report_3 = gr.Textbox(
                            label="Culture Report 3 (optional)",
                            placeholder="Date: 2026-01-20\n...",
                            lines=8,
                        )

                btn_analyse_manual = gr.Button("🔬 Analyse Reports", variant="primary")
                manual_error = gr.HTML(value="")
                manual_patient_html = gr.HTML(value="")
                manual_clinician_html = gr.HTML(value="")

                def on_analyse_manual(r1, r2, r3):
                    texts = [t for t in [r1, r2, r3] if t and t.strip()]
                    if len(texts) < 2:
                        return (
                            "<p style='color:#c0392b'>Please enter at least 2 culture reports.</p>",
                            "",
                            "",
                        )

                    reports = []
                    for text in texts:
                        try:
                            report = extract_structured_data(text)
                            # Ensure raw_text is cleared before passing downstream
                            reports.append(CultureReport(
                                date=report.date,
                                organism=report.organism,
                                cfu=report.cfu,
                                resistance_markers=report.resistance_markers,
                                specimen_type=report.specimen_type,
                                contamination_flag=report.contamination_flag,
                                raw_text="",  # SAFETY: never forwarded to MedGemma
                            ))
                        except ExtractionError as e:
                            return (
                                f"<p style='color:#c0392b'>Extraction error: {e}</p>",
                                "",
                                "",
                            )

                    try:
                        patient_out, clinician_out = run_pipeline(reports)
                        patient_html, clinician_html = format_output_html(patient_out, clinician_out)
                        return "", patient_html, clinician_html
                    except Exception as e:
                        return (
                            f"<p style='color:#c0392b'>Analysis error: {e}</p>",
                            "",
                            "",
                        )

                btn_analyse_manual.click(
                    fn=on_analyse_manual,
                    inputs=[report_1, report_2, report_3],
                    outputs=[manual_error, manual_patient_html, manual_clinician_html],
                )

    return demo

In [ ]:
# Launch the CultureSense Gradio app
demo = build_gradio_app(model, tokenizer, is_stub)
demo.launch(share=True)

---

## Safety & Regulatory Positioning

- **No output** from any module, in any mode, shall contain a named diagnosis.
- Confidence scores are capped at **0.95** (never 1.0 — clinical epistemic humility).
- Both output modes end with **hardcoded disclaimer text** that cannot be overridden.
- MedGemma is **never prompted with raw user text** — only structured JSON.
- A post-processing safety scan using `BANNED_DIAGNOSTIC_PHRASES` provides a second layer of defence.

> *This notebook is a Kaggle competition prototype only. It is not intended for clinical use,
> does not constitute medical advice, and has not been evaluated for diagnostic accuracy.*
